In [ ]:
# soilgrids_fetcher.py
#
# Description:
# This script fetches soil property data from the SoilGrids v2 REST API
# for a single geographic location (latitude, longitude) and prints the
# raw data as a pandas DataFrame.
#
# Usage:
# 1. Install required packages:
#    pip install requests pandas numpy
#
# 2. Run from the command line:
#    python soilgrids_fetcher.py

import logging
import sys
from typing import List

import numpy as np
import pandas as pd
import requests

# --- Configuration Constants ---

# Standard SoilGrids depths to query
SOILGRIDS_DEPTHS = ["0-5cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm", "100-200cm"]

# Properties to request from the SoilGrids API
SOILGRIDS_PROPERTIES = ["sand", "silt", "clay", "bdod", "soc", "phh2o", "cec"]


def fetch_soilgrids_data(latitude: float, longitude: float, properties: List[str], depths: List[str]) -> pd.DataFrame:
    """
    Fetches soil property data from the SoilGrids v2 REST API for a given location.

    Args:
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.
        properties (List[str]): List of soil properties to fetch.
        depths (List[str]): List of soil depths to fetch.

    Returns:
        pd.DataFrame: A DataFrame with soil properties as columns and depths as rows.

    Raises:
        requests.exceptions.RequestException: If a network error occurs.
        ValueError: If the API returns a non-200 status or malformed data.
    """
    logging.info(f"Fetching SoilGrids data for lat={latitude}, lon={longitude}")
    base_url = "https://rest.isric.org/soilgrids/v2.0/properties/query"
    params = {
        'lon': longitude,
        'lat': latitude,
        'property': properties,
        'depth': depths,
        'value': 'mean'
    }

    try:
        response = requests.get(base_url, params=params, timeout=30)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    except requests.exceptions.RequestException as e:
        logging.error(f"Network error fetching SoilGrids data: {e}")
        logging.error("Could not connect to the SoilGrids server. Please check your internet connection.")
        raise

    data = response.json()
    layers = data.get('properties', {}).get('layers', [])
    if not layers:
        raise ValueError("SoilGrids API returned no layer data.")

    # Parse the nested JSON into a more usable format
    parsed_data = {}
    for layer in layers:
        layer_name = layer['name']
        for depth_data in layer['depths']:
            depth_label = depth_data['label']
            if depth_label not in parsed_data:
                parsed_data[depth_label] = {'depth': depth_label}
            # The API may return None for values, which we handle later
            parsed_data[depth_label][layer_name] = depth_data['values']['mean']

    if not parsed_data:
        raise ValueError("Could not parse layer data from SoilGrids response.")

    df = pd.DataFrame.from_dict(parsed_data, orient='index')
    # Replace any Python 'None' values with NumPy's Not a Number (NaN)
    df = df.replace({None: np.nan})

    # Ensure all requested properties are present as columns, even if all are NaN
    for prop in properties:
        if prop not in df.columns:
            df[prop] = np.nan
            
    # Reorder DataFrame to match the requested depths
    df = df.reindex(depths)
    
    return df

# --- Command-Line Interface ---
if __name__ == "__main__":
    # Set up basic logging to print messages to the console
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', stream=sys.stdout)

    # Example coordinates for Punjab Agricultural University, Ludhiana, India
    EXAMPLE_LAT = 12.3811 
    EXAMPLE_LON = 78.9366

    try:
        # 1. Fetch data from SoilGrids
        soil_data_df = fetch_soilgrids_data(
            latitude=EXAMPLE_LAT,
            longitude=EXAMPLE_LON,
            properties=SOILGRIDS_PROPERTIES,
            depths=SOILGRIDS_DEPTHS
        )
        
        # 2. Print the resulting DataFrame
        print("\n--- Raw SoilGrids Data ---")
        print(soil_data_df)
        print("\nSuccessfully fetched data.")

    except (requests.exceptions.RequestException, ValueError) as e:
        logging.error(f"Script failed: {e}")

In [ ]:
# soil_dssat_processor_robust.py
#
# Description:
# This script fetches a comprehensive set of soil properties from the SoilGrids v2 REST API.
# It now fetches properties one-by-one to avoid overwhelming the server and causing a 500 error.
# It then processes them into DSSAT-compatible soil parameters and exports to a CSV file.
#
# Usage:
# 1. Install required packages:
#    pip install requests pandas numpy
#
# 2. Run from the command line:
#    python soil_dssat_processor_robust.py

import logging
import sys
from typing import List
import time

import numpy as np
import pandas as pd
import requests

# --- Configuration Constants ---
SOILGRIDS_DEPTHS = ["0-5cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm", "100-200cm"]
SOILGRIDS_PROPERTIES = [
    "sand", "silt", "clay", "bdod", "cfvo", "wv0033", "wv1500", "ksat",
    "soc", "phh2o", "cec", "nitrogen", "phos", "k"
]
PARTICLE_DENSITY_G_CM3 = 2.65

def fetch_soilgrids_data_robust(latitude: float, longitude: float, properties: List[str], depths: List[str]) -> pd.DataFrame:
    """
    Fetches soil property data robustly by querying one property at a time
    to prevent server-side errors from complex requests.
    """
    logging.info(f"Fetching SoilGrids data for lat={latitude}, lon={longitude}")
    base_url = "https://rest.isric.org/soilgrids/v2.0/properties/query"
    
    all_properties_data = [] # To store list of DataFrames, one for each property

    for prop in properties:
        logging.info(f"--> Fetching property: {prop}")
        params = {
            'lon': longitude,
            'lat': latitude,
            'property': prop,
            'depth': depths,
            'value': 'mean'
        }
        
        try:
            response = requests.get(base_url, params=params, timeout=45)
            response.raise_for_status()
            
            data = response.json()
            layers = data.get('properties', {}).get('layers', [])
            if not layers:
                logging.warning(f"No layer data returned for property: {prop}. Skipping.")
                continue

            # This parsing logic is simplified as we only have one property per call
            parsed_data = {}
            for depth_data in layers[0]['depths']:
                depth_label = depth_data['label']
                mean_value = depth_data['values']['mean']
                parsed_data[depth_label] = mean_value
            
            # Create a DataFrame for the current property
            prop_df = pd.DataFrame.from_dict(parsed_data, orient='index', columns=[prop])
            all_properties_data.append(prop_df)

            time.sleep(0.2) # Small delay to be courteous to the API server

        except requests.exceptions.RequestException as e:
            logging.error(f"Failed to fetch data for property '{prop}': {e}. It will be missing from the output.")
        except (ValueError, KeyError) as e:
            logging.error(f"Failed to parse data for property '{prop}': {e}. Skipping.")

    if not all_properties_data:
        raise ValueError("Failed to fetch any data from SoilGrids.")

    # Concatenate all individual property DataFrames into a single DataFrame
    final_df = pd.concat(all_properties_data, axis=1)
    final_df = final_df.reindex(depths)
    
    # Ensure all requested properties are present as columns, even if all are NaN
    for prop in properties:
        if prop not in final_df.columns:
            final_df[prop] = np.nan
            
    return final_df


def process_to_dssat_format(raw_df: pd.DataFrame) -> pd.DataFrame:
    """
    Converts raw SoilGrids data into a DSSAT-formatted DataFrame.
    """
    logging.info("Processing raw data into DSSAT format...")
    dssat_df = pd.DataFrame(index=raw_df.index)

    dssat_df['SLB'] = [int(label.split('-')[1].replace('cm', '')) for label in raw_df.index]
    dssat_df['SBDM'] = raw_df['bdod'] / 100
    dssat_df['SLCL'] = raw_df['clay'] / 10
    dssat_df['SLSI'] = raw_df['silt'] / 10
    dssat_df['SLCF'] = raw_df['cfvo'] / 10
    dssat_df['SLLL'] = raw_df['wv1500'] / 1000
    dssat_df['SDUL'] = raw_df['wv0033'] / 1000
    dssat_df['SSKS'] = (10**raw_df['ksat']) / 24
    dssat_df['SSAT'] = 1 - (dssat_df['SBDM'] / PARTICLE_DENSITY_G_CM3)
    rgf_map = {'0-5cm': 1.0, '5-15cm': 1.0, '15-30cm': 1.0, '30-60cm': 0.8, '60-100cm': 0.5, '100-200cm': 0.2}
    dssat_df['SRGF'] = dssat_df.index.map(rgf_map)
    dssat_df['SLOC'] = raw_df['soc'] / 100
    dssat_df['SLHW'] = raw_df['phh2o'] / 10
    dssat_df['SCEC'] = raw_df['cec'] / 10
    dssat_df['SLNI'] = raw_df['nitrogen'] / 1000
    dssat_df['SLPX'] = raw_df['phos']
    dssat_df['SLKE'] = raw_df['k'] * 391
    
    return dssat_df.round(3)


# --- Command-Line Interface ---
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', stream=sys.stdout)

    EXAMPLE_LAT = 12.3811
    EXAMPLE_LON = 78.9366
    OUTPUT_CSV_FILE = "dssat_soil_profile.csv"

    try:
        # 1. Fetch raw data robustly
        raw_soil_data = fetch_soilgrids_data_robust(
            latitude=EXAMPLE_LAT,
            longitude=EXAMPLE_LON,
            properties=SOILGRIDS_PROPERTIES,
            depths=SOILGRIDS_DEPTHS
        )
        print("\n--- Raw SoilGrids Data (with different units) ---")
        print(raw_soil_data)

        # 2. Process data into DSSAT format
        dssat_soil_profile = process_to_dssat_format(raw_soil_data)
        print("\n\n--- Processed DSSAT Soil Profile Data ---")
        print(dssat_soil_profile)
        
        # 3. Export to CSV
        dssat_soil_profile.to_csv(OUTPUT_CSV_FILE)
        logging.info(f"\nSuccessfully exported DSSAT-formatted data to '{OUTPUT_CSV_FILE}'")

    except (requests.exceptions.RequestException, ValueError) as e:
        logging.error(f"Script failed: {e}")

In [ ]:
# soil_dssat_processor_robust.py
#
# Description:
# This script fetches a comprehensive set of soil properties from the SoilGrids v2 REST API.
# It now fetches properties one-by-one to avoid overwhelming the server and causing a 500 error.
# It then processes them into DSSAT-compatible soil parameters and exports to a CSV file.
#
# Usage:
# 1. Install required packages:
#    pip install requests pandas numpy
#
# 2. Run from the command line:
#    python soil_dssat_processor_robust.py

import logging
import sys
from typing import List
import time

import numpy as np
import pandas as pd
import requests

# --- Configuration Constants ---
SOILGRIDS_DEPTHS = ["0-5cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm", "100-200cm"]
SOILGRIDS_PROPERTIES = [
    "sand", "silt", "clay", "bdod", "cfvo", "wv0033", "wv1500", "ksat",
    "soc", "phh2o", "cec", "nitrogen", "phos", "k"
]
PARTICLE_DENSITY_G_CM3 = 2.65

def fetch_soilgrids_data_robust(latitude: float, longitude: float, properties: List[str], depths: List[str]) -> pd.DataFrame:
    """
    Fetches soil property data robustly by querying one property at a time
    to prevent server-side errors from complex requests.
    """
    logging.info(f"Fetching SoilGrids data for lat={latitude}, lon={longitude}")
    base_url = "https://rest.isric.org/soilgrids/v2.0/properties/query"
    
    all_properties_data = [] # To store list of DataFrames, one for each property

    for prop in properties:
        logging.info(f"--> Fetching property: {prop}")
        params = {
            'lon': longitude,
            'lat': latitude,
            'property': prop,
            'depth': depths,
            'value': 'mean'
        }
        
        try:
            response = requests.get(base_url, params=params, timeout=45)
            response.raise_for_status()
            
            data = response.json()
            layers = data.get('properties', {}).get('layers', [])
            if not layers:
                logging.warning(f"No layer data returned for property: {prop}. Skipping.")
                continue

            # This parsing logic is simplified as we only have one property per call
            parsed_data = {}
            for depth_data in layers[0]['depths']:
                depth_label = depth_data['label']
                mean_value = depth_data['values']['mean']
                parsed_data[depth_label] = mean_value
            
            # Create a DataFrame for the current property
            prop_df = pd.DataFrame.from_dict(parsed_data, orient='index', columns=[prop])
            all_properties_data.append(prop_df)

            time.sleep(0.2) # Small delay to be courteous to the API server

        except requests.exceptions.RequestException as e:
            logging.error(f"Failed to fetch data for property '{prop}': {e}. It will be missing from the output.")
        except (ValueError, KeyError) as e:
            logging.error(f"Failed to parse data for property '{prop}': {e}. Skipping.")

    if not all_properties_data:
        raise ValueError("Failed to fetch any data from SoilGrids.")

    # Concatenate all individual property DataFrames into a single DataFrame
    final_df = pd.concat(all_properties_data, axis=1)
    final_df = final_df.reindex(depths)
    
    # Ensure all requested properties are present as columns, even if all are NaN
    for prop in properties:
        if prop not in final_df.columns:
            final_df[prop] = np.nan
            
    return final_df


def process_to_dssat_format(raw_df: pd.DataFrame) -> pd.DataFrame:
    """
    Converts raw SoilGrids data into a DSSAT-formatted DataFrame.
    """
    logging.info("Processing raw data into DSSAT format...")
    dssat_df = pd.DataFrame(index=raw_df.index)

    dssat_df['SLB'] = [int(label.split('-')[1].replace('cm', '')) for label in raw_df.index]
    dssat_df['SBDM'] = raw_df['bdod'] / 100
    dssat_df['SLCL'] = raw_df['clay'] / 10
    dssat_df['SLSI'] = raw_df['silt'] / 10
    dssat_df['SLCF'] = raw_df['cfvo'] / 10
    dssat_df['SLLL'] = raw_df['wv1500'] / 1000
    dssat_df['SDUL'] = raw_df['wv0033'] / 1000
    dssat_df['SSKS'] = (10**raw_df['ksat']) / 24
    dssat_df['SSAT'] = 1 - (dssat_df['SBDM'] / PARTICLE_DENSITY_G_CM3)
    rgf_map = {'0-5cm': 1.0, '5-15cm': 1.0, '15-30cm': 1.0, '30-60cm': 0.8, '60-100cm': 0.5, '100-200cm': 0.2}
    dssat_df['SRGF'] = dssat_df.index.map(rgf_map)
    dssat_df['SLOC'] = raw_df['soc'] / 100
    dssat_df['SLHW'] = raw_df['phh2o'] / 10
    dssat_df['SCEC'] = raw_df['cec'] / 10
    dssat_df['SLNI'] = raw_df['nitrogen'] / 1000
    dssat_df['SLPX'] = raw_df['phos']
    dssat_df['SLKE'] = raw_df['k'] * 391
    
    return dssat_df.round(3)


# --- Command-Line Interface ---
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', stream=sys.stdout)

    EXAMPLE_LAT = 12.3811
    EXAMPLE_LON = 78.9366
    OUTPUT_CSV_FILE = "dssat_soil_profile.csv"

    try:
        # 1. Fetch raw data robustly
        raw_soil_data = fetch_soilgrids_data_robust(
            latitude=EXAMPLE_LAT,
            longitude=EXAMPLE_LON,
            properties=SOILGRIDS_PROPERTIES,
            depths=SOILGRIDS_DEPTHS
        )
        print("\n--- Raw SoilGrids Data (with different units) ---")
        print(raw_soil_data)

        # 2. Process data into DSSAT format
        dssat_soil_profile = process_to_dssat_format(raw_soil_data)
        print("\n\n--- Processed DSSAT Soil Profile Data ---")
        print(dssat_soil_profile)
        
        # 3. Export to CSV
        dssat_soil_profile.to_csv(OUTPUT_CSV_FILE)
        logging.info(f"\nSuccessfully exported DSSAT-formatted data to '{OUTPUT_CSV_FILE}'")

    except (requests.exceptions.RequestException, ValueError) as e:
        logging.error(f"Script failed: {e}")

In [ ]:
# soil_dssat_processor_robust.py
#
# Description:
# This script fetches available soil properties from the SoilGrids v2 REST API.
# It now fetches properties one-by-one to avoid overwhelming the server and causing a 500 error.
# It then processes them into DSSAT-compatible soil parameters and exports to a CSV file.
#
# Usage:
# 1. Install required packages:
#    pip install requests pandas numpy
#
# 2. Run from the command line:
#    python soil_dssat_processor_robust.py

import logging
import sys
from typing import List
import time

import numpy as np
import pandas as pd
import requests

# --- Configuration Constants ---
SOILGRIDS_DEPTHS = ["0-5cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm", "100-200cm"]

# MODIFIED: Removed ksat, phos, and k, which were causing server errors.
SOILGRIDS_PROPERTIES = [
    "sand", "silt", "clay", "bdod", "cfvo", "wv0033", "wv1500",
    "soc", "phh2o", "cec", "nitrogen"
]
PARTICLE_DENSITY_G_CM3 = 2.65

def fetch_soilgrids_data_robust(latitude: float, longitude: float, properties: List[str], depths: List[str]) -> pd.DataFrame:
    """
    Fetches soil property data robustly by querying one property at a time
    to prevent server-side errors from complex requests.
    """
    logging.info(f"Fetching SoilGrids data for lat={latitude}, lon={longitude}")
    base_url = "https://rest.isric.org/soilgrids/v2.0/properties/query"
    
    all_properties_data = []

    for prop in properties:
        logging.info(f"--> Fetching property: {prop}")
        params = {
            'lon': longitude,
            'lat': latitude,
            'property': prop,
            'depth': depths,
            'value': 'mean'
        }
        
        try:
            response = requests.get(base_url, params=params, timeout=45)
            response.raise_for_status()
            
            data = response.json()
            layers = data.get('properties', {}).get('layers', [])
            if not layers:
                logging.warning(f"No layer data returned for property: {prop}. Skipping.")
                continue

            parsed_data = {}
            for depth_data in layers[0]['depths']:
                depth_label = depth_data['label']
                mean_value = depth_data['values']['mean']
                parsed_data[depth_label] = mean_value
            
            prop_df = pd.DataFrame.from_dict(parsed_data, orient='index', columns=[prop])
            all_properties_data.append(prop_df)

            time.sleep(0.2)

        except requests.exceptions.RequestException as e:
            logging.error(f"Failed to fetch data for property '{prop}': {e}. It will be missing from the output.")
        except (ValueError, KeyError) as e:
            logging.error(f"Failed to parse data for property '{prop}': {e}. Skipping.")

    if not all_properties_data:
        raise ValueError("Failed to fetch any data from SoilGrids.")

    final_df = pd.concat(all_properties_data, axis=1)
    final_df = final_df.reindex(depths)
    
    for prop in properties:
        if prop not in final_df.columns:
            final_df[prop] = np.nan
            
    return final_df


def process_to_dssat_format(raw_df: pd.DataFrame) -> pd.DataFrame:
    """
    Converts raw SoilGrids data into a DSSAT-formatted DataFrame.
    """
    logging.info("Processing raw data into DSSAT format...")
    dssat_df = pd.DataFrame(index=raw_df.index)

    dssat_df['SLB'] = [int(label.split('-')[1].replace('cm', '')) for label in raw_df.index]
    dssat_df['SBDM'] = raw_df['bdod'] / 100
    dssat_df['SLCL'] = raw_df['clay'] / 10
    dssat_df['SLSI'] = raw_df['silt'] / 10
    dssat_df['SLCF'] = raw_df['cfvo'] / 10
    dssat_df['SLLL'] = raw_df['wv1500'] / 1000
    dssat_df['SDUL'] = raw_df['wv0033'] / 1000
    
    # SSKS is no longer available
    dssat_df['SSKS'] = np.nan 
    
    dssat_df['SSAT'] = 1 - (dssat_df['SBDM'] / PARTICLE_DENSITY_G_CM3)
    rgf_map = {'0-5cm': 1.0, '5-15cm': 1.0, '15-30cm': 1.0, '30-60cm': 0.8, '60-100cm': 0.5, '100-200cm': 0.2}
    dssat_df['SRGF'] = dssat_df.index.map(rgf_map)
    dssat_df['SLOC'] = raw_df['soc'] / 100
    dssat_df['SLHW'] = raw_df['phh2o'] / 10
    dssat_df['SCEC'] = raw_df['cec'] / 10
    dssat_df['SLNI'] = raw_df['nitrogen'] / 1000
    
    # SLPX and SLKE are no longer available
    dssat_df['SLPX'] = np.nan
    dssat_df['SLKE'] = np.nan
    
    return dssat_df.round(3)


# --- Command-Line Interface ---
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', stream=sys.stdout)

    EXAMPLE_LAT = 12.3811
    EXAMPLE_LON = 78.9366
    OUTPUT_CSV_FILE = "dssat_soil_profile.csv"

    try:
        raw_soil_data = fetch_soilgrids_data_robust(
            latitude=EXAMPLE_LAT,
            longitude=EXAMPLE_LON,
            properties=SOILGRIDS_PROPERTIES,
            depths=SOILGRIDS_DEPTHS
        )
        print("\n--- Raw SoilGrids Data (with different units) ---")
        print(raw_soil_data)

        dssat_soil_profile = process_to_dssat_format(raw_soil_data)
        print("\n\n--- Processed DSSAT Soil Profile Data ---")
        print(dssat_soil_profile)
        
        dssat_soil_profile.to_csv(OUTPUT_CSV_FILE)
        logging.info(f"\nSuccessfully exported DSSAT-formatted data to '{OUTPUT_CSV_FILE}'")

    except (requests.exceptions.RequestException, ValueError) as e:
        logging.error(f"Script failed: {e}")

Available Parameters
Here is the list of the remaining SoilGrids parameters that the script fetches and the corresponding DSSAT parameter names they are converted into:

bdod ➡️ SBDM (Bulk Density)

clay ➡️ SLCL (Clay Percentage)

silt ➡️ SLSI (Silt Percentage)

cfvo ➡️ SLCF (Coarse Fragments)

wv1500 ➡️ SLLL (Wilting Point)

wv0033 ➡️ SDUL (Field Capacity)

soc ➡️ SLOC (Soil Organic Carbon)

phh2o ➡️ SLHW (pH in Water)

cec ➡️ SCEC (Cation Exchange Capacity)

nitrogen ➡️ SLNI (Total Nitrogen)

Of course. Here is a clear breakdown of the DSSAT parameters your script can generate versus those it cannot.

***
## ✅ Available from Your Script

Your Python script successfully fetches data from SoilGrids and uses it to generate the following core soil profile properties:

* **`SLB`**: Layer Base Depth
* **`SBDM`**: Bulk Density
* **`SLCL`**: Clay Percentage
* **`SLSI`**: Silt Percentage
* **`SLCF`**: Coarse Fragments
* **`SLLL`**: Wilting Point
* **`SDUL`**: Field Capacity
* **`SSAT`**: Saturation Level
* **`SRGF`**: Root Growth Factor
* **`SLOC`**: Soil Organic Carbon
* **`SLHW`**: pH in Water
* **`SCEC`**: Cation Exchange Capacity
* **`SLNI`**: Total Nitrogen (*Note: DSSAT often needs initial mineral nitrogen instead*)

***
## ❌ Not Available / Must Be Added Manually

These are the essential parameters that SoilGrids does not provide or that failed to download. You must add these to your soil file from other sources (like soil test reports, lookup tables, or expert estimates).

* **`SSKS`**: Saturated Hydraulic Conductivity
* **`SLPX`**: Extractable Phosphorus (P)
* **`SLKE`**: Exchangeable Potassium (K)
* **`SALB`**: Soil Albedo (reflectivity)
* **`SLU1`**: Runoff Curve Number
* **`SLDR`**: Soil Drainage Rate
* **Initial Conditions**: The starting amount of water (`SH2O`), nitrate (`SLNI` as NO₃), and ammonium (`SNH4`) in the soil for your specific simulation.

In [ ]:
# soil_dssat_processor_rosetta.py
#
# Description:
# This script fetches available soil properties from the SoilGrids v2 REST API.
# It then processes them into DSSAT-compatible soil parameters.
# Finally, it uses the rosetta-soil library to estimate Saturated
# Hydraulic Conductivity (SSKS) from the soil texture data.
#
# Usage:
# 1. Install required packages:
#    pip install requests pandas numpy rosetta-soil
#
# 2. Run from the command line:
#    python soil_dssat_processor_rosetta.py

import logging
import sys
import time
from typing import List

import numpy as np
import pandas as pd
import requests
from rosetta import rosetta  # Import the rosetta library

# --- Configuration Constants ---
SOILGRIDS_DEPTHS = ["0-5cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm", "100-200cm"]

# MODIFIED: Removed ksat, phos, and k, which are not available from the server.
SOILGRIDS_PROPERTIES = [
    "sand", "silt", "clay", "bdod", "cfvo", "wv0033", "wv1500",
    "soc", "phh2o", "cec", "nitrogen"
]
PARTICLE_DENSITY_G_CM3 = 2.65


def fetch_soilgrids_data_robust(latitude: float, longitude: float, properties: List[str], depths: List[str]) -> pd.DataFrame:
    """
    Fetches soil property data robustly by querying one property at a time
    to prevent server-side errors from complex requests.
    """
    logging.info(f"Fetching SoilGrids data for lat={latitude}, lon={longitude}")
    base_url = "https://rest.isric.org/soilgrids/v2.0/properties/query"

    all_properties_data = []

    for prop in properties:
        logging.info(f"--> Fetching property: {prop}")
        params = {
            'lon': longitude,
            'lat': latitude,
            'property': prop,
            'depth': depths,
            'value': 'mean'
        }

        try:
            response = requests.get(base_url, params=params, timeout=45)
            response.raise_for_status()

            data = response.json()
            layers = data.get('properties', {}).get('layers', [])
            if not layers:
                logging.warning(f"No layer data returned for property: {prop}. Skipping.")
                continue

            parsed_data = {}
            for depth_data in layers[0]['depths']:
                depth_label = depth_data['label']
                mean_value = depth_data.get('values', {}).get('mean')
                if mean_value is None:
                    logging.warning(f"Mean value not found for property {prop} at depth {depth_label}. Setting to NaN.")
                    mean_value = np.nan
                parsed_data[depth_label] = mean_value
                
            prop_df = pd.DataFrame.from_dict(parsed_data, orient='index', columns=[prop])
            all_properties_data.append(prop_df)

            time.sleep(0.2)

        except requests.exceptions.RequestException as e:
            logging.error(f"Failed to fetch data for property '{prop}': {e}. It will be missing from the output.")
        except (ValueError, KeyError) as e:
            logging.error(f"Failed to parse data for property '{prop}': {e}. Skipping.")

    if not all_properties_data:
        raise ValueError("Failed to fetch any data from SoilGrids.")

    final_df = pd.concat(all_properties_data, axis=1)
    final_df = final_df.reindex(depths)

    for prop in properties:
        if prop not in final_df.columns:
            final_df[prop] = np.nan

    return final_df


def process_to_dssat_format(raw_df: pd.DataFrame) -> pd.DataFrame:
    """
    Converts raw SoilGrids data into a DSSAT-formatted DataFrame.
    """
    logging.info("Processing raw data into DSSAT format...")
    dssat_df = pd.DataFrame(index=raw_df.index)

    # Check for missing essential columns before proceeding
    required_cols = ['bdod', 'clay', 'silt', 'cfvo', 'wv1500', 'wv0033', 'soc', 'phh2o', 'cec', 'nitrogen']
    for col in required_cols:
        if col not in raw_df or raw_df[col].isnull().all():
            logging.warning(f"Essential data column '{col}' is missing or empty. Corresponding DSSAT values will be NaN.")
            raw_df[col] = np.nan # Ensure column exists to prevent KeyErrors

    dssat_df['SLB'] = [int(label.split('-')[1].replace('cm', '')) for label in raw_df.index]
    dssat_df['SBDM'] = raw_df['bdod'] / 100
    dssat_df['SLCL'] = raw_df['clay'] / 10
    dssat_df['SLSI'] = raw_df['silt'] / 10
    dssat_df['SLCF'] = raw_df['cfvo'] / 10
    dssat_df['SLLL'] = raw_df['wv1500'] / 1000
    dssat_df['SDUL'] = raw_df['wv0033'] / 1000
    
    # Placeholder for SSKS - will be calculated later with Rosetta
    dssat_df['SSKS'] = np.nan
    
    dssat_df['SSAT'] = 1 - (dssat_df['SBDM'] / PARTICLE_DENSITY_G_CM3)
    rgf_map = {'0-5cm': 1.0, '5-15cm': 1.0, '15-30cm': 1.0, '30-60cm': 0.8, '60-100cm': 0.5, '100-200cm': 0.2}
    dssat_df['SRGF'] = dssat_df.index.map(rgf_map)
    dssat_df['SLOC'] = raw_df['soc'] / 100
    dssat_df['SLHW'] = raw_df['phh2o'] / 10
    dssat_df['SCEC'] = raw_df['cec'] / 10
    dssat_df['SLNI'] = raw_df['nitrogen'] / 1000
    
    # SLPX and SLKE are not available from SoilGrids
    dssat_df['SLPX'] = np.nan
    dssat_df['SLKE'] = np.nan
    
    return dssat_df

def calculate_hydraulic_properties_with_rosetta(dssat_df: pd.DataFrame) -> pd.DataFrame:
    """
    Uses the Rosetta model to calculate SSKS from soil texture and bulk density.
    """
    logging.info("Calculating hydraulic properties with Rosetta model...")

    # Create a copy to avoid SettingWithCopyWarning
    df = dssat_df.copy()

    # 1. Calculate Sand % since Rosetta needs it as an input
    df['SLSA'] = 100 - df['SLCL'] - df['SLSI']
    
    # Handle potential NaNs in input columns
    input_cols = ['SLSA', 'SLSI', 'SLCL', 'SBDM']
    if df[input_cols].isnull().values.any():
        logging.warning("Missing texture or bulk density data. Cannot run Rosetta. SSKS will be NaN.")
        df['SSKS'] = np.nan
        df = df.drop(columns=['SLSA'])
        return df

    # 2. Prepare the input data for the Rosetta model
    # It requires a list of lists: [sand, silt, clay, bulk_density]
    rosetta_input = df[input_cols].values.tolist()
    
    try:
        # 3. Run Rosetta v3, which uses Sand, Silt, Clay, and Bulk Density
        # The output is a list of tuples: (theta_r, theta_s, alpha, n, Ksat)
        hydraulic_props = rosetta(3, rosetta_input)
        
        # 4. Extract Saturated Hydraulic Conductivity (Ksat)
        # The result is the 5th element (index 4) and is in cm/day.
        # DSSAT's SSKS requires cm/hr, so we divide by 24.
        df['SSKS'] = [prop[4] / 24 for prop in hydraulic_props]
        
    except Exception as e:
        logging.error(f"Rosetta model failed: {e}. SSKS will be NaN.")
        df['SSKS'] = np.nan

    # 5. Clean up the temporary sand column
    df = df.drop(columns=['SLSA'])
    
    return df


# --- Command-Line Interface ---
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', stream=sys.stdout)

    EXAMPLE_LAT = 12.3811
    EXAMPLE_LON = 78.9366
    OUTPUT_CSV_FILE = "dssat_soil_profile.csv"

    try:
        # 1. Fetch available data from SoilGrids
        raw_soil_data = fetch_soilgrids_data_robust(
            latitude=EXAMPLE_LAT,
            longitude=EXAMPLE_LON,
            properties=SOILGRIDS_PROPERTIES,
            depths=SOILGRIDS_DEPTHS
        )
        print("\n--- Raw SoilGrids Data (with different units) ---")
        print(raw_soil_data.to_string())

        # 2. Process data into base DSSAT format
        dssat_soil_profile = process_to_dssat_format(raw_soil_data)
        
        # 3. Calculate SSKS using Rosetta model
        dssat_soil_profile_final = calculate_hydraulic_properties_with_rosetta(dssat_soil_profile)
        
        print("\n\n--- Processed DSSAT Soil Profile Data (with Rosetta SSKS) ---")
        print(dssat_soil_profile_final.round(3).to_string())
        
        # 4. Export to CSV
        dssat_soil_profile_final.round(3).to_csv(OUTPUT_CSV_FILE)
        logging.info(f"\nSuccessfully exported DSSAT-formatted data to '{OUTPUT_CSV_FILE}'")

    except (requests.exceptions.RequestException, ValueError) as e:
        logging.error(f"Script failed: {e}")

In [ ]:
# soil_dssat_processor_rosetta_FIXED.py
#
# Description:
# This script fetches available soil properties from the SoilGrids v2 REST API,
# handling server rate-limiting with a robust retry mechanism. It then processes
# the data into DSSAT-compatible soil parameters and uses the rosetta-soil
# library to estimate Saturated Hydraulic Conductivity (SSKS).
#
# Usage:
# 1. Install required packages:
#    pip install requests pandas numpy rosetta-soil
#
# 2. Run from the command line:
#    python soil_dssat_processor_rosetta_FIXED.py

import logging
import sys
import time
from typing import List

import numpy as np
import pandas as pd
import requests
from rosetta import rosetta  # Import the rosetta library

# --- Configuration Constants ---
SOILGRIDS_DEPTHS = ["0-5cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm", "100-200cm"]
SOILGRIDS_PROPERTIES = [
    "sand", "silt", "clay", "bdod", "cfvo", "wv0033", "wv1500",
    "soc", "phh2o", "cec", "nitrogen"
]
PARTICLE_DENSITY_G_CM3 = 2.65


def fetch_soilgrids_data_robust(latitude: float, longitude: float, properties: List[str], depths: List[str]) -> pd.DataFrame:
    """
    Fetches soil property data robustly by querying one property at a time
    and implementing a retry mechanism with backoff for 429 errors.
    """
    logging.info(f"Fetching SoilGrids data for lat={latitude}, lon={longitude}")
    base_url = "https://rest.isric.org/soilgrids/v2.0/properties/query"
    all_properties_data = []

    # --- Retry Logic Configuration ---
    max_retries = 4
    initial_retry_delay_seconds = 5 # Start with a 5-second wait

    for prop in properties:
        logging.info(f"--> Fetching property: {prop}")
        params = {
            'lon': longitude, 'lat': latitude, 'property': prop,
            'depth': depths, 'value': 'mean'
        }

        # --- Start of Retry Loop ---
        for attempt in range(max_retries):
            try:
                response = requests.get(base_url, params=params, timeout=45)
                # Check specifically for 429 error to handle it
                if response.status_code == 429:
                    # Raise an HTTPError so the except block can catch it
                    response.raise_for_status()

                # If any other error occurs (e.g., 500), it will be caught below
                response.raise_for_status()

                # --- Success Case ---
                data = response.json()
                layers = data.get('properties', {}).get('layers', [])
                if not layers:
                    logging.warning(f"No layer data returned for property: {prop}. Skipping.")
                    break # Exit retry loop, move to next property

                parsed_data = {}
                for depth_data in layers[0]['depths']:
                    depth_label = depth_data['label']
                    mean_value = depth_data.get('values', {}).get('mean')
                    if mean_value is None:
                        logging.warning(f"Mean value not found for {prop} at {depth_label}. Setting to NaN.")
                        mean_value = np.nan
                    parsed_data[depth_label] = mean_value

                prop_df = pd.DataFrame.from_dict(parsed_data, orient='index', columns=[prop])
                all_properties_data.append(prop_df)

                time.sleep(1) # Add a small courtesy delay after a successful request
                break # Success! Exit the retry loop.

            except requests.exceptions.HTTPError as e:
                # --- Failure Case (Retry Logic) ---
                if e.response.status_code == 429:
                    if attempt < max_retries - 1:
                        # Increase wait time for each failed attempt (exponential backoff)
                        wait_time = initial_retry_delay_seconds * (2 ** attempt)
                        logging.warning(f"Rate limit hit for '{prop}' (Attempt {attempt + 1}/{max_retries}). Retrying in {wait_time} seconds...")
                        time.sleep(wait_time)
                    else:
                        logging.error(f"Failed to fetch data for '{prop}' after {max_retries} attempts due to rate limiting: {e}.")
                else:
                    # Handle other HTTP errors (e.g., 500 Internal Server Error)
                    logging.error(f"HTTP error for '{prop}': {e}. It will be missing from the output.")
                    break # Don't retry on non-429 errors

            except requests.exceptions.RequestException as e:
                logging.error(f"Failed to fetch data for '{prop}': {e}. It will be missing from the output.")
                break # Stop trying for this property on general request errors
            except (ValueError, KeyError) as e:
                logging.error(f"Failed to parse data for '{prop}': {e}. Skipping.")
                break # Stop trying if parsing fails

    if not all_properties_data:
        raise ValueError("Failed to fetch any data from SoilGrids.")

    final_df = pd.concat(all_properties_data, axis=1)
    final_df = final_df.reindex(depths)

    for prop in properties:
        if prop not in final_df.columns:
            final_df[prop] = np.nan
    return final_df


def process_to_dssat_format(raw_df: pd.DataFrame) -> pd.DataFrame:
    """
    Converts raw SoilGrids data into a DSSAT-formatted DataFrame.
    """
    logging.info("Processing raw data into DSSAT format...")
    dssat_df = pd.DataFrame(index=raw_df.index)

    # Check for missing essential columns before proceeding
    required_cols = ['bdod', 'clay', 'silt', 'cfvo', 'wv1500', 'wv0033', 'soc', 'phh2o', 'cec', 'nitrogen']
    for col in required_cols:
        if col not in raw_df or raw_df[col].isnull().all():
            logging.warning(f"Essential data column '{col}' is missing or empty. Corresponding DSSAT values will be NaN.")
            raw_df[col] = np.nan # Ensure column exists to prevent KeyErrors

    dssat_df['SLB'] = [int(label.split('-')[1].replace('cm', '')) for label in raw_df.index]
    dssat_df['SBDM'] = raw_df['bdod'] / 100
    dssat_df['SLCL'] = raw_df['clay'] / 10
    dssat_df['SLSI'] = raw_df['silt'] / 10
    dssat_df['SLCF'] = raw_df['cfvo'] / 10
    dssat_df['SLLL'] = raw_df['wv1500'] / 1000
    dssat_df['SDUL'] = raw_df['wv0033'] / 1000
    
    # Placeholder for SSKS - will be calculated later with Rosetta
    dssat_df['SSKS'] = np.nan
    
    dssat_df['SSAT'] = 1 - (dssat_df['SBDM'] / PARTICLE_DENSITY_G_CM3)
    rgf_map = {'0-5cm': 1.0, '5-15cm': 1.0, '15-30cm': 1.0, '30-60cm': 0.8, '60-100cm': 0.5, '100-200cm': 0.2}
    dssat_df['SRGF'] = dssat_df.index.map(rgf_map)
    dssat_df['SLOC'] = raw_df['soc'] / 100
    dssat_df['SLHW'] = raw_df['phh2o'] / 10
    dssat_df['SCEC'] = raw_df['cec'] / 10
    dssat_df['SLNI'] = raw_df['nitrogen'] / 1000
    
    # SLPX and SLKE are not available from SoilGrids
    dssat_df['SLPX'] = np.nan
    dssat_df['SLKE'] = np.nan
    
    return dssat_df

def calculate_hydraulic_properties_with_rosetta(dssat_df: pd.DataFrame) -> pd.DataFrame:
    """
    Uses the Rosetta model to calculate SSKS from soil texture and bulk density.
    """
    logging.info("Calculating hydraulic properties with Rosetta model...")

    # Create a copy to avoid SettingWithCopyWarning
    df = dssat_df.copy()

    # 1. Calculate Sand % since Rosetta needs it as an input
    df['SLSA'] = 100 - df['SLCL'] - df['SLSI']
    
    # Handle potential NaNs in input columns
    input_cols = ['SLSA', 'SLSI', 'SLCL', 'SBDM']
    if df[input_cols].isnull().values.any():
        logging.warning("Missing texture or bulk density data. Cannot run Rosetta. SSKS will be NaN.")
        df['SSKS'] = np.nan
        df = df.drop(columns=['SLSA'])
        return df

    # 2. Prepare the input data for the Rosetta model
    # It requires a list of lists: [sand, silt, clay, bulk_density]
    rosetta_input = df[input_cols].values.tolist()
    
    try:
        # 3. Run Rosetta v3, which uses Sand, Silt, Clay, and Bulk Density
        # The output is a list of tuples: (theta_r, theta_s, alpha, n, Ksat)
        hydraulic_props = rosetta(3, rosetta_input)
        
        # 4. Extract Saturated Hydraulic Conductivity (Ksat)
        # The result is the 5th element (index 4) and is in cm/day.
        # DSSAT's SSKS requires cm/hr, so we divide by 24.
        df['SSKS'] = [prop[4] / 24 for prop in hydraulic_props]
        
    except Exception as e:
        logging.error(f"Rosetta model failed: {e}. SSKS will be NaN.")
        df['SSKS'] = np.nan

    # 5. Clean up the temporary sand column
    df = df.drop(columns=['SLSA'])
    
    return df


# --- Command-Line Interface ---
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', stream=sys.stdout)

    EXAMPLE_LAT = 12.3811
    EXAMPLE_LON = 78.9366
    OUTPUT_CSV_FILE = "dssat_soil_profile.csv"

    try:
        # 1. Fetch available data from SoilGrids
        raw_soil_data = fetch_soilgrids_data_robust(
            latitude=EXAMPLE_LAT,
            longitude=EXAMPLE_LON,
            properties=SOILGRIDS_PROPERTIES,
            depths=SOILGRIDS_DEPTHS
        )
        print("\n--- Raw SoilGrids Data (with different units) ---")
        print(raw_soil_data.to_string())

        # 2. Process data into base DSSAT format
        dssat_soil_profile = process_to_dssat_format(raw_soil_data)
        
        # 3. Calculate SSKS using Rosetta model
        dssat_soil_profile_final = calculate_hydraulic_properties_with_rosetta(dssat_soil_profile)
        
        print("\n\n--- Processed DSSAT Soil Profile Data (with Rosetta SSKS) ---")
        print(dssat_soil_profile_final.round(3).to_string())
        
        # 4. Export to CSV
        dssat_soil_profile_final.round(3).to_csv(OUTPUT_CSV_FILE)
        logging.info(f"\nSuccessfully exported DSSAT-formatted data to '{OUTPUT_CSV_FILE}'")

    except (requests.exceptions.RequestException, ValueError) as e:
        logging.error(f"Script failed: {e}")

In [ ]:
# soil_dssat_processor_rosetta_FINAL.py
#
# Description:
# This script fetches soil properties from SoilGrids, processes them into
# DSSAT format, and uses the rosetta-soil library to estimate hydraulic
# properties, correctly handling API rate limits and data types.
#
# Usage:
# 1. Install required packages:
#    pip install requests pandas numpy rosetta-soil
#
# 2. Run from the command line:
#    python soil_dssat_processor_rosetta_FINAL.py

import logging
import sys
import time
from typing import List

import numpy as np
import pandas as pd
import requests
# MODIFIED: Import both rosetta and the required SoilData class
from rosetta import rosetta, SoilData

# --- Configuration Constants ---
SOILGRIDS_DEPTHS = ["0-5cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm", "100-200cm"]
SOILGRIDS_PROPERTIES = [
    "sand", "silt", "clay", "bdod", "cfvo", "wv0033", "wv1500",
    "soc", "phh2o", "cec", "nitrogen"
]
PARTICLE_DENSITY_G_CM3 = 2.65


def fetch_soilgrids_data_robust(latitude: float, longitude: float, properties: List[str], depths: List[str]) -> pd.DataFrame:
    """
    Fetches soil property data robustly by querying one property at a time
    and implementing a retry mechanism with backoff for 429 errors.
    """
    logging.info(f"Fetching SoilGrids data for lat={latitude}, lon={longitude}")
    base_url = "https://rest.isric.org/soilgrids/v2.0/properties/query"
    all_properties_data = []
    max_retries = 4
    initial_retry_delay_seconds = 5

    for prop in properties:
        logging.info(f"--> Fetching property: {prop}")
        params = {
            'lon': longitude, 'lat': latitude, 'property': prop,
            'depth': depths, 'value': 'mean'
        }

        for attempt in range(max_retries):
            try:
                response = requests.get(base_url, params=params, timeout=45)
                if response.status_code == 429:
                    response.raise_for_status()
                response.raise_for_status()

                data = response.json()
                layers = data.get('properties', {}).get('layers', [])
                if not layers:
                    logging.warning(f"No layer data returned for property: {prop}. Skipping.")
                    break

                parsed_data = {}
                for depth_data in layers[0]['depths']:
                    depth_label = depth_data['label']
                    mean_value = depth_data.get('values', {}).get('mean')
                    if mean_value is None:
                        mean_value = np.nan
                    parsed_data[depth_label] = mean_value

                prop_df = pd.DataFrame.from_dict(parsed_data, orient='index', columns=[prop])
                all_properties_data.append(prop_df)
                time.sleep(1)
                break

            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 429 and attempt < max_retries - 1:
                    wait_time = initial_retry_delay_seconds * (2 ** attempt)
                    logging.warning(f"Rate limit hit for '{prop}'. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    logging.error(f"HTTP error for '{prop}': {e}. It will be missing.")
                    break
            except requests.exceptions.RequestException as e:
                logging.error(f"Failed to fetch data for '{prop}': {e}. It will be missing.")
                break
            except (ValueError, KeyError) as e:
                logging.error(f"Failed to parse data for '{prop}': {e}. Skipping.")
                break

    if not all_properties_data:
        raise ValueError("Failed to fetch any data from SoilGrids.")

    final_df = pd.concat(all_properties_data, axis=1).reindex(depths)
    for prop in properties:
        if prop not in final_df.columns:
            final_df[prop] = np.nan
    return final_df


def process_to_dssat_format(raw_df: pd.DataFrame) -> pd.DataFrame:
    """
    Converts raw SoilGrids data into a DSSAT-formatted DataFrame.
    """
    logging.info("Processing raw data into DSSAT format...")
    dssat_df = pd.DataFrame(index=raw_df.index)

    required_cols = ['bdod', 'clay', 'silt', 'cfvo', 'wv1500', 'wv0033', 'soc', 'phh2o', 'cec', 'nitrogen']
    for col in required_cols:
        if col not in raw_df or raw_df[col].isnull().all():
            logging.warning(f"Essential data column '{col}' is missing. DSSAT values will be NaN.")
            raw_df[col] = np.nan

    dssat_df['SLB'] = [int(label.split('-')[1].replace('cm', '')) for label in raw_df.index]
    dssat_df['SBDM'] = raw_df['bdod'] / 100
    dssat_df['SLCL'] = raw_df['clay'] / 10
    dssat_df['SLSI'] = raw_df['silt'] / 10
    dssat_df['SLCF'] = raw_df['cfvo'] / 10
    dssat_df['SLLL'] = raw_df['wv1500'] / 1000
    dssat_df['SDUL'] = raw_df['wv0033'] / 1000
    dssat_df['SSKS'] = np.nan
    dssat_df['SSAT'] = 1 - (dssat_df['SBDM'] / PARTICLE_DENSITY_G_CM3)
    rgf_map = {'0-5cm': 1.0, '5-15cm': 1.0, '15-30cm': 1.0, '30-60cm': 0.8, '60-100cm': 0.5, '100-200cm': 0.2}
    dssat_df['SRGF'] = dssat_df.index.map(rgf_map)
    dssat_df['SLOC'] = raw_df['soc'] / 100
    dssat_df['SLHW'] = raw_df['phh2o'] / 10
    dssat_df['SCEC'] = raw_df['cec'] / 10
    dssat_df['SLNI'] = raw_df['nitrogen'] / 1000
    dssat_df['SLPX'] = np.nan
    dssat_df['SLKE'] = np.nan
    return dssat_df

def calculate_hydraulic_properties_with_rosetta(dssat_df: pd.DataFrame) -> pd.DataFrame:
    """
    Uses the Rosetta model to calculate SSKS, formatting the input
    correctly using the SoilData class.
    """
    logging.info("Calculating hydraulic properties with Rosetta model...")
    df = dssat_df.copy()

    df['SLSA'] = 100 - df['SLCL'] - df['SLSI']
    
    input_cols = ['SLSA', 'SLSI', 'SLCL', 'SBDM']
    if df[input_cols].isnull().values.any():
        logging.warning("Missing texture or bulk density data. Cannot run Rosetta. SSKS will be NaN.")
        df['SSKS'] = np.nan
        return df.drop(columns=['SLSA'])

    try:
        # MODIFIED: Create a SoilData instance as required by the library
        soil_data = SoilData(
            sand=df['SLSA'].tolist(),
            silt=df['SLSI'].tolist(),
            clay=df['SLCL'].tolist(),
            bd=df['SBDM'].tolist()
        )
        
        # MODIFIED: Pass the SoilData object to the rosetta function
        hydraulic_props = rosetta(3, soil_data)
        
        # Ksat is the 5th element (index 4) in cm/day.
        # DSSAT requires cm/hr, so we divide by 24.
        df['SSKS'] = [prop[4] / 24 for prop in hydraulic_props]
        
    except Exception as e:
        logging.error(f"Rosetta model failed: {e}. SSKS will be NaN.")
        df['SSKS'] = np.nan

    return df.drop(columns=['SLSA'])


# --- Command-Line Interface ---
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', stream=sys.stdout)

    EXAMPLE_LAT = 12.3811
    EXAMPLE_LON = 78.9366
    OUTPUT_CSV_FILE = "dssat_soil_profile.csv"

    try:
        raw_soil_data = fetch_soilgrids_data_robust(
            latitude=EXAMPLE_LAT,
            longitude=EXAMPLE_LON,
            properties=SOILGRIDS_PROPERTIES,
            depths=SOILGRIDS_DEPTHS
        )
        print("\n--- Raw SoilGrids Data (with different units) ---")
        print(raw_soil_data.to_string())

        dssat_soil_profile = process_to_dssat_format(raw_soil_data)
        dssat_soil_profile_final = calculate_hydraulic_properties_with_rosetta(dssat_soil_profile)
        
        print("\n\n--- Processed DSSAT Soil Profile Data (with Rosetta SSKS) ---")
        print(dssat_soil_profile_final.round(3).to_string())
        
        dssat_soil_profile_final.round(3).to_csv(OUTPUT_CSV_FILE)
        logging.info(f"\nSuccessfully exported DSSAT-formatted data to '{OUTPUT_CSV_FILE}'")

    except (requests.exceptions.RequestException, ValueError) as e:
        logging.error(f"Script failed: {e}")

In [ ]:
import logging
import sys
import time
from typing import List

import numpy as np
import pandas as pd
import requests
from rosetta import rosetta, SoilData

# --- 1. SET YOUR INPUTS HERE ---
#---------------------------------
LATITUDE = 12.3811
LONGITUDE = 78.9366
OUTPUT_FILENAME = "MY_GENERATED_SOIL.SOL"
#---------------------------------


# --- 2. HELPER FUNCTIONS (No changes needed here) ---

# --- Configuration Constants ---
SOILGRIDS_DEPTHS = ["0-5cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm", "100-200cm"]
SOILGRIDS_PROPERTIES = [
    "sand", "silt", "clay", "bdod", "cfvo", "wv0033", "wv1500",
    "soc", "phh2o", "cec", "nitrogen"
]
PARTICLE_DENSITY_G_CM3 = 2.65

def fetch_soilgrids_data_robust(latitude: float, longitude: float, properties: List[str], depths: List[str]) -> pd.DataFrame:
    """Fetches soil property data robustly with a retry mechanism."""
    logging.info(f"Fetching SoilGrids data for lat={latitude}, lon={longitude}")
    base_url = "https://rest.isric.org/soilgrids/v2.0/properties/query"
    all_properties_data = []
    max_retries = 4
    initial_retry_delay_seconds = 5

    for prop in properties:
        logging.info(f"--> Fetching property: {prop}")
        params = {'lon': longitude, 'lat': latitude, 'property': prop, 'depth': depths, 'value': 'mean'}
        for attempt in range(max_retries):
            try:
                response = requests.get(base_url, params=params, timeout=45)
                if response.status_code == 429:
                    response.raise_for_status()
                response.raise_for_status()
                data = response.json()
                layers = data.get('properties', {}).get('layers', [])
                if not layers:
                    logging.warning(f"No layer data for {prop}. Skipping.")
                    break
                parsed_data = {
                    depth_data['label']: depth_data.get('values', {}).get('mean', np.nan)
                    for depth_data in layers[0]['depths']
                }
                prop_df = pd.DataFrame.from_dict(parsed_data, orient='index', columns=[prop])
                all_properties_data.append(prop_df)
                time.sleep(1)
                break
            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 429 and attempt < max_retries - 1:
                    wait_time = initial_retry_delay_seconds * (2 ** attempt)
                    logging.warning(f"Rate limit hit for '{prop}'. Retrying in {wait_time}s...")
                    time.sleep(wait_time)
                else:
                    logging.error(f"HTTP error for '{prop}': {e}. It will be missing.")
                    break
            except (requests.exceptions.RequestException, ValueError, KeyError) as e:
                logging.error(f"Failed on property '{prop}': {e}. Skipping.")
                break
    if not all_properties_data:
        raise ValueError("Failed to fetch any data from SoilGrids.")
    final_df = pd.concat(all_properties_data, axis=1).reindex(depths)
    for prop in properties:
        if prop not in final_df.columns:
            final_df[prop] = np.nan
    return final_df

def process_to_dssat_format(raw_df: pd.DataFrame) -> pd.DataFrame:
    """Converts raw SoilGrids data into a DSSAT-formatted DataFrame."""
    logging.info("Processing raw data into DSSAT format...")
    dssat_df = pd.DataFrame(index=raw_df.index)
    for col in SOILGRIDS_PROPERTIES:
        if col not in raw_df or raw_df[col].isnull().all():
            logging.warning(f"Essential data column '{col}' is missing. DSSAT values will be NaN.")
            raw_df[col] = np.nan
    dssat_df['SLB'] = [int(label.split('-')[1].replace('cm', '')) for label in raw_df.index]
    dssat_df['SBDM'] = raw_df['bdod'] / 100
    dssat_df['SLCL'] = raw_df['clay'] / 10
    dssat_df['SLSI'] = raw_df['silt'] / 10
    dssat_df['SLCF'] = raw_df['cfvo'] / 10
    dssat_df['SLLL'] = raw_df['wv1500'] / 1000
    dssat_df['SDUL'] = raw_df['wv0033'] / 1000
    dssat_df['SSKS'] = np.nan
    dssat_df['SSAT'] = 1 - (dssat_df['SBDM'] / PARTICLE_DENSITY_G_CM3)
    rgf_map = {'0-5cm': 1.0, '5-15cm': 1.0, '15-30cm': 1.0, '30-60cm': 0.8, '60-100cm': 0.5, '100-200cm': 0.2}
    dssat_df['SRGF'] = dssat_df.index.map(rgf_map)
    dssat_df['SLOC'] = raw_df['soc'] / 100
    dssat_df['SLHW'] = raw_df['phh2o'] / 10
    dssat_df['SCEC'] = raw_df['cec'] / 10
    dssat_df['SLNI'] = raw_df['nitrogen'] / 1000
    dssat_df['SLPX'] = np.nan
    dssat_df['SLKE'] = np.nan
    return dssat_df

def calculate_hydraulic_properties_with_rosetta(dssat_df: pd.DataFrame) -> pd.DataFrame:
    """Uses the Rosetta model to calculate SSKS."""
    logging.info("Calculating hydraulic properties with Rosetta model...")
    df = dssat_df.copy()
    df['SLSA'] = 100 - df['SLCL'] - df['SLSI']
    input_cols = ['SLSA', 'SLSI', 'SLCL', 'SBDM']
    if df[input_cols].isnull().values.any():
        logging.warning("Missing texture or bulk density. Cannot run Rosetta. SSKS will be NaN.")
        df['SSKS'] = np.nan
        return df.drop(columns=['SLSA'])
    try:
        soil_data = SoilData(sand=df['SLSA'].tolist(), silt=df['SLSI'].tolist(), clay=df['SLCL'].tolist(), bd=df['SBDM'].tolist())
        hydraulic_props = rosetta(3, soil_data)
        df['SSKS'] = [prop[4] / 24 for prop in hydraulic_props]
    except Exception as e:
        logging.error(f"Rosetta model failed: {e}. SSKS will be NaN.")
        df['SSKS'] = np.nan
    return df.drop(columns=['SLSA'])

def get_texture_class(sand: float, silt: float, clay: float) -> str:
    """Determines USDA soil texture class from sand, silt, clay percentages."""
    if (silt + 1.5 * clay) < 15: return 'S'
    if (silt + 1.5 * clay) >= 15 and (silt + 2 * clay) < 30: return 'LS'
    if (clay >= 7 and clay < 20) and (sand > 52) and ((silt + 2 * clay) >= 30): return 'SL'
    if (clay < 7) and (silt < 50) and ((silt + 2 * clay) >= 30): return 'SL'
    if (clay >= 7 and clay < 27) and (silt >= 28 and silt < 50) and (sand <= 52): return 'L'
    if (silt >= 50 and clay >= 12 and clay < 27): return 'SIL'
    if (silt >= 50 and clay < 12): return 'SI'
    if (silt >= 80 and clay < 12): return 'SI'
    if (clay >= 20 and clay < 35) and (silt < 28) and (sand > 45): return 'SCL'
    if (clay >= 27 and clay < 40) and (sand > 20 and sand <= 45): return 'CL'
    if (clay >= 27 and clay < 40) and (sand <= 20): return 'SICL'
    if (clay >= 35) and (sand > 45): return 'SC'
    if (clay >= 40) and (silt >= 40): return 'SIC'
    if (clay >= 40) and (sand <= 45) and (silt < 40): return 'C'
    return 'UNKNOWN'

def write_sol_file(df: pd.DataFrame, lat: float, lon: float, filename: str):
    """Writes the DataFrame to a DSSAT-compatible .SOL file."""
    logging.info(f"Writing DSSAT .SOL file to {filename}")
    top_layer = df.iloc[0]
    sand_pct = 100 - top_layer.get('SLCL', 0) - top_layer.get('SLSI', 0)
    texture = get_texture_class(sand_pct, top_layer.get('SLSI', 0), top_layer.get('SLCL', 0))
    soil_id = f"SG{str(int(abs(lat*100)))+str(int(abs(lon*100))):<10}"
    with open(filename, 'w') as f:
        f.write("*SOILS: DSSAT Soil Input File from SoilGrids Data\n!\n")
        f.write(f"! Auto-generated for location Lat={lat:.4f}, Lon={lon:.4f}\n!\n\n")
        f.write(f"*{soil_id} SOILGRIDS   {texture:<5s} {int(df['SLB'].max()):>5} SoilGrids Auto-Generated Profile\n")
        f.write("@SITE        COUNTRY          LAT     LONG SCS FAMILY\n")
        f.write(f" SoilGrids                {lat:>8.2f} {lon:>8.2f} Generic\n")
        f.write("@ SCOM  SALB  SLU1  SLDR  SLRO  SLNF  SLPF  SMHB  SMPX  SMKE\n")
        f.write("   -99  0.13   6.0   -99  75.0  1.00  1.00   -99   -99   -99\n")
        f.write("@  SLB  SLMH  SLLL  SDUL  SSAT  SRGF  SSKS  SBDM  SLOC  SLCL  SLSI  SLCF  SLNI  SLHW  SLHB  SCEC  SADC\n")
        for _, row in df.iterrows():
            row = row.fillna(-99)
            f.write(f"{row['SLB']:>6.0f} {-99:>5.0f} {row['SLLL']:>6.3f} {row['SDUL']:>6.3f} {row['SSAT']:>6.3f} {row['SRGF']:>6.3f} ")
            f.write(f"{row['SSKS']:>6.3f} {row['SBDM']:>6.3f} {row['SLOC']:>6.3f} {row['SLCL']:>6.1f} {row['SLSI']:>6.1f} ")
            f.write(f"{row['SLCF']:>6.1f} {row['SLNI']:>6.3f} {row['SLHW']:>6.1f} {-99:>5.0f} {row['SCEC']:>6.1f} {-99:>5.0f}\n")


# --- 3. MAIN EXECUTION LOGIC ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', stream=sys.stdout)

try:
    raw_soil_data = fetch_soilgrids_data_robust(LATITUDE, LONGITUDE, SOILGRIDS_PROPERTIES, SOILGRIDS_DEPTHS)
    dssat_profile = process_to_dssat_format(raw_soil_data)
    final_dssat_profile = calculate_hydraulic_properties_with_rosetta(dssat_profile)

    print("\n--- Final Processed DSSAT Data ---")
    print(final_dssat_profile.round(3).to_string())

    write_sol_file(final_dssat_profile, LATITUDE, LONGITUDE, OUTPUT_FILENAME)
    print(f"\n✅ Successfully created soil file: {OUTPUT_FILENAME}")

except (requests.exceptions.RequestException, ValueError) as e:
    logging.error(f"Script failed: {e}")

In [ ]:
import logging
import sys
import time
from typing import List

import numpy as np
import pandas as pd
import requests
from rosetta import rosetta, SoilData

# --- 1. SET YOUR INPUTS HERE ---
#---------------------------------
LATITUDE = 12.3811
LONGITUDE = 78.9366
OUTPUT_FILENAME = "MY_GENERATED_SOIL.SOL"
#---------------------------------


# --- 2. HELPER FUNCTIONS ---

# --- Configuration Constants ---
SOILGRIDS_DEPTHS = ["0-5cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm", "100-200cm"]
SOILGRIDS_PROPERTIES = [
    "sand", "silt", "clay", "bdod", "cfvo", "wv0033", "wv1500",
    "soc", "phh2o", "cec", "nitrogen"
]
PARTICLE_DENSITY_G_CM3 = 2.65

def fetch_soilgrids_data_robust(latitude: float, longitude: float, properties: List[str], depths: List[str]) -> pd.DataFrame:
    """Fetches soil property data robustly with a retry mechanism."""
    logging.info(f"Fetching SoilGrids data for lat={latitude}, lon={longitude}")
    base_url = "https://rest.isric.org/soilgrids/v2.0/properties/query"
    all_properties_data = []
    max_retries = 4
    initial_retry_delay_seconds = 5

    for prop in properties:
        logging.info(f"--> Fetching property: {prop}")
        params = {'lon': longitude, 'lat': latitude, 'property': prop, 'depth': depths, 'value': 'mean'}
        for attempt in range(max_retries):
            try:
                response = requests.get(base_url, params=params, timeout=45)
                if response.status_code == 429:
                    response.raise_for_status()
                response.raise_for_status()
                data = response.json()
                layers = data.get('properties', {}).get('layers', [])
                if not layers:
                    logging.warning(f"No layer data for {prop}. Skipping.")
                    break
                parsed_data = {
                    depth_data['label']: depth_data.get('values', {}).get('mean', np.nan)
                    for depth_data in layers[0]['depths']
                }
                prop_df = pd.DataFrame.from_dict(parsed_data, orient='index', columns=[prop])
                all_properties_data.append(prop_df)
                time.sleep(1)
                break
            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 429 and attempt < max_retries - 1:
                    wait_time = initial_retry_delay_seconds * (2 ** attempt)
                    logging.warning(f"Rate limit hit for '{prop}'. Retrying in {wait_time}s...")
                    time.sleep(wait_time)
                else:
                    logging.error(f"HTTP error for '{prop}': {e}. It will be missing.")
                    break
            except (requests.exceptions.RequestException, ValueError, KeyError) as e:
                logging.error(f"Failed on property '{prop}': {e}. Skipping.")
                break
    if not all_properties_data:
        raise ValueError("Failed to fetch any data from SoilGrids.")
    final_df = pd.concat(all_properties_data, axis=1).reindex(depths)
    for prop in properties:
        if prop not in final_df.columns:
            final_df[prop] = np.nan
    return final_df

def process_to_dssat_format(raw_df: pd.DataFrame) -> pd.DataFrame:
    """Converts raw SoilGrids data into a DSSAT-formatted DataFrame."""
    logging.info("Processing raw data into DSSAT format...")
    dssat_df = pd.DataFrame(index=raw_df.index)
    for col in SOILGRIDS_PROPERTIES:
        if col not in raw_df or raw_df[col].isnull().all():
            logging.warning(f"Essential data column '{col}' is missing. DSSAT values will be NaN.")
            raw_df[col] = np.nan
    
    # SLB  - Layer Base Depth (cm)
    dssat_df['SLB'] = [int(label.split('-')[1].replace('cm', '')) for label in raw_df.index]
    # SBDM - Bulk Density, Moist (g/cm³)
    dssat_df['SBDM'] = raw_df['bdod'] / 100
    # SLCL - Clay Content (%)
    dssat_df['SLCL'] = raw_df['clay'] / 10
    # SLSI - Silt Content (%)
    dssat_df['SLSI'] = raw_df['silt'] / 10
    # SLCF - Coarse Fraction (>2mm) (%)
    dssat_df['SLCF'] = raw_df['cfvo'] / 10
    # SLLL - Lower Limit (Wilting Point) (cm³/cm³)
    dssat_df['SLLL'] = raw_df['wv1500'] / 1000
    # SDUL - Drained Upper Limit (Field Capacity) (cm³/cm³)
    dssat_df['SDUL'] = raw_df['wv0033'] / 1000
    # SSKS - Saturated Hydraulic Conductivity (cm/h)
    dssat_df['SSKS'] = np.nan
    # SSAT - Saturated Water Content (Porosity) (cm³/cm³)
    dssat_df['SSAT'] = 1 - (dssat_df['SBDM'] / PARTICLE_DENSITY_G_CM3)
    # SRGF - Root Growth Factor (0-1)
    rgf_map = {'0-5cm': 1.0, '5-15cm': 1.0, '15-30cm': 1.0, '30-60cm': 0.8, '60-100cm': 0.5, '100-200cm': 0.2}
    dssat_df['SRGF'] = dssat_df.index.map(rgf_map)
    # SLOC - Organic Carbon (%)
    dssat_df['SLOC'] = raw_df['soc'] / 100
    # SLHW - pH in Water
    dssat_df['SLHW'] = raw_df['phh2o'] / 10
    # SCEC - Cation Exchange Capacity (cmol/kg)
    dssat_df['SCEC'] = raw_df['cec'] / 10
    # SLNI - Total Nitrogen (%)
    dssat_df['SLNI'] = raw_df['nitrogen'] / 1000
    # SLPX - Phosphorus (not available)
    dssat_df['SLPX'] = np.nan
    # SLKE - Potassium (not available)
    dssat_df['SLKE'] = np.nan
    
    return dssat_df

def calculate_hydraulic_properties_with_rosetta(dssat_df: pd.DataFrame) -> pd.DataFrame:
    """Uses the Rosetta model to calculate SSKS."""
    logging.info("Calculating hydraulic properties with Rosetta model...")
    df = dssat_df.copy()
    df['SLSA'] = 100 - df['SLCL'] - df['SLSI']
    input_cols = ['SLSA', 'SLSI', 'SLCL', 'SBDM']
    if df[input_cols].isnull().values.any():
        logging.warning("Missing texture or bulk density. Cannot run Rosetta. SSKS will be NaN.")
        df['SSKS'] = np.nan
        return df.drop(columns=['SLSA'])
    try:
        soil_data = SoilData(sand=df['SLSA'].tolist(), silt=df['SLSI'].tolist(), clay=df['SLCL'].tolist(), bd=df['SBDM'].tolist())
        hydraulic_props = rosetta(3, soil_data)
        df['SSKS'] = [prop[4] / 24 for prop in hydraulic_props]
    except Exception as e:
        logging.error(f"Rosetta model failed: {e}. SSKS will be NaN.")
        df['SSKS'] = np.nan
    return df.drop(columns=['SLSA'])

def get_texture_class(sand: float, silt: float, clay: float) -> str:
    """Determines USDA soil texture class from sand, silt, clay percentages."""
    if (silt + 1.5 * clay) < 15: return 'S'
    if (silt + 1.5 * clay) >= 15 and (silt + 2 * clay) < 30: return 'LS'
    if (clay >= 7 and clay < 20) and (sand > 52) and ((silt + 2 * clay) >= 30): return 'SL'
    if (clay < 7) and (silt < 50) and ((silt + 2 * clay) >= 30): return 'SL'
    if (clay >= 7 and clay < 27) and (silt >= 28 and silt < 50) and (sand <= 52): return 'L'
    if (silt >= 50 and clay >= 12 and clay < 27): return 'SIL'
    if (silt >= 50 and clay < 12): return 'SI'
    if (silt >= 80 and clay < 12): return 'SI'
    if (clay >= 20 and clay < 35) and (silt < 28) and (sand > 45): return 'SCL'
    if (clay >= 27 and clay < 40) and (sand > 20 and sand <= 45): return 'CL'
    if (clay >= 27 and clay < 40) and (sand <= 20): return 'SICL'
    if (clay >= 35) and (sand > 45): return 'SC'
    if (clay >= 40) and (silt >= 40): return 'SIC'
    if (clay >= 40) and (sand <= 45) and (silt < 40): return 'C'
    return 'UNKNOWN'

def write_sol_file(df: pd.DataFrame, lat: float, lon: float, filename: str):
    """Writes the DataFrame to a DSSAT-compatible .SOL file."""
    logging.info(f"Writing DSSAT .SOL file to {filename}")
    top_layer = df.iloc[0]
    sand_pct = 100 - top_layer.get('SLCL', 0) - top_layer.get('SLSI', 0)
    texture = get_texture_class(sand_pct, top_layer.get('SLSI', 0), top_layer.get('SLCL', 0))
    soil_id = f"SG{str(int(abs(lat*100)))+str(int(abs(lon*100))):<10}"
    
    with open(filename, 'w') as f:
        f.write("*SOILS: DSSAT Soil Input File from SoilGrids Data\n!\n")
        f.write(f"! Auto-generated for location Lat={lat:.4f}, Lon={lon:.4f}\n!\n\n")
        f.write(f"*{soil_id} SOILGRIDS   {texture:<5s} {int(df['SLB'].max()):>5} SoilGrids Auto-Generated Profile\n")
        f.write("@SITE        COUNTRY          LAT     LONG SCS FAMILY\n")
        f.write(f" SoilGrids                {lat:>8.2f} {lon:>8.2f} Generic\n")

        # --- DSSAT Master Parameter Glossary ---
        # SCOM - Soil Color (Not used)
        # SALB - Soil Albedo (0-1)
        # SLU1 - Stage 1 Evaporation Limit (mm)
        # SLDR - Soil Drainage Rate
        # SLRO - Runoff Curve Number
        # SLNF - Mineralization Factor (0-1)
        # SLPF - Photosynthesis Factor (0-1)
        # SMHB, SMPX, SMKE - Not used, placeholders for soil mineralogy
        f.write("@ SCOM  SALB  SLU1  SLDR  SLRO  SLNF  SLPF  SMHB  SMPX  SMKE\n")
        f.write("   -99  0.13   6.0   -99  75.0  1.00  1.00   -99   -99   -99\n")
        
        # --- DSSAT Soil Layer Parameter Glossary ---
        # SLB  - Layer Base Depth (cm)
        # SLMH - Master Horizon (-99 for not specified)
        # SLLL - Lower Limit (Wilting Point) (cm³/cm³)
        # SDUL - Drained Upper Limit (Field Capacity) (cm³/cm³)
        # SSAT - Saturated Water Content (Porosity) (cm³/cm³)
        # SRGF - Root Growth Factor (0-1)
        # SSKS - Saturated Hydraulic Conductivity (cm/h)
        # SBDM - Bulk Density, Moist (g/cm³)
        # SLOC - Organic Carbon (%)
        # SLCL - Clay Content (%)
        # SLSI - Silt Content (%)
        # SLCF - Coarse Fraction (>2mm) (%)
        # SLNI - Total Nitrogen (%)
        # SLHW - pH in Water
        # SLHB - Horizon Base (-99 for not specified)
        # SCEC - Cation Exchange Capacity (cmol/kg)
        # SADC - Anion Adsorption Coefficient (-99 for not specified)
        f.write("@  SLB  SLMH  SLLL  SDUL  SSAT  SRGF  SSKS  SBDM  SLOC  SLCL  SLSI  SLCF  SLNI  SLHW  SLHB  SCEC  SADC\n")
        
        for _, row in df.iterrows():
            row = row.fillna(-99)
            f.write(f"{row['SLB']:>6.0f} {-99:>5.0f} {row['SLLL']:>6.3f} {row['SDUL']:>6.3f} {row['SSAT']:>6.3f} {row['SRGF']:>6.3f} ")
            f.write(f"{row['SSKS']:>6.3f} {row['SBDM']:>6.3f} {row['SLOC']:>6.3f} {row['SLCL']:>6.1f} {row['SLSI']:>6.1f} ")
            f.write(f"{row['SLCF']:>6.1f} {row['SLNI']:>6.3f} {row['SLHW']:>6.1f} {-99:>5.0f} {row['SCEC']:>6.1f} {-99:>5.0f}\n")


# --- 3. MAIN EXECUTION LOGIC ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', stream=sys.stdout)

try:
    raw_soil_data = fetch_soilgrids_data_robust(LATITUDE, LONGITUDE, SOILGRIDS_PROPERTIES, SOILGRIDS_DEPTHS)
    dssat_profile = process_to_dssat_format(raw_soil_data)
    final_dssat_profile = calculate_hydraulic_properties_with_rosetta(dssat_profile)

    print("\n--- Final Processed DSSAT Data ---")
    print(final_dssat_profile.round(3).to_string())

    write_sol_file(final_dssat_profile, LATITUDE, LONGITUDE, OUTPUT_FILENAME)
    print(f"\n✅ Successfully created soil file: {OUTPUT_FILENAME}")

except (requests.exceptions.RequestException, ValueError) as e:
    logging.error(f"Script failed: {e}")

In [ ]:
import logging
import sys
import time
from typing import List

import numpy as np
import pandas as pd
import requests
from rosetta import rosetta, SoilData

# --- 1. SET YOUR INPUTS HERE ---
#---------------------------------
LATITUDE = 12.3811
LONGITUDE = 78.9366
COUNTRY = "India"  # <-- ADDED: Set the country for the .SOL file metadata
OUTPUT_FILENAME = "MY_GENERATED_SOIL.SOL"
#---------------------------------


# --- 2. HELPER FUNCTIONS (No changes needed here) ---

# --- Configuration Constants ---
SOILGRIDS_DEPTHS = ["0-5cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm", "100-200cm"]
SOILGRIDS_PROPERTIES = [
    "sand", "silt", "clay", "bdod", "cfvo", "wv0033", "wv1500",
    "soc", "phh2o", "cec", "nitrogen"
]
PARTICLE_DENSITY_G_CM3 = 2.65

def fetch_soilgrids_data_robust(latitude: float, longitude: float, properties: List[str], depths: List[str]) -> pd.DataFrame:
    """Fetches soil property data robustly with a retry mechanism."""
    logging.info(f"Fetching SoilGrids data for lat={latitude}, lon={longitude}")
    base_url = "https://rest.isric.org/soilgrids/v2.0/properties/query"
    all_properties_data = []
    max_retries = 4
    initial_retry_delay_seconds = 5

    for prop in properties:
        logging.info(f"--> Fetching property: {prop}")
        params = {'lon': longitude, 'lat': latitude, 'property': prop, 'depth': depths, 'value': 'mean'}
        for attempt in range(max_retries):
            try:
                response = requests.get(base_url, params=params, timeout=45)
                if response.status_code == 429:
                    response.raise_for_status()
                response.raise_for_status()
                data = response.json()
                layers = data.get('properties', {}).get('layers', [])
                if not layers:
                    logging.warning(f"No layer data for {prop}. Skipping.")
                    break
                parsed_data = {
                    depth_data['label']: depth_data.get('values', {}).get('mean', np.nan)
                    for depth_data in layers[0]['depths']
                }
                prop_df = pd.DataFrame.from_dict(parsed_data, orient='index', columns=[prop])
                all_properties_data.append(prop_df)
                time.sleep(1)
                break
            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 429 and attempt < max_retries - 1:
                    wait_time = initial_retry_delay_seconds * (2 ** attempt)
                    logging.warning(f"Rate limit hit for '{prop}'. Retrying in {wait_time}s...")
                    time.sleep(wait_time)
                else:
                    logging.error(f"HTTP error for '{prop}': {e}. It will be missing.")
                    break
            except (requests.exceptions.RequestException, ValueError, KeyError) as e:
                logging.error(f"Failed on property '{prop}': {e}. Skipping.")
                break
    if not all_properties_data:
        raise ValueError("Failed to fetch any data from SoilGrids.")
    final_df = pd.concat(all_properties_data, axis=1).reindex(depths)
    for prop in properties:
        if prop not in final_df.columns:
            final_df[prop] = np.nan
    return final_df

def process_to_dssat_format(raw_df: pd.DataFrame) -> pd.DataFrame:
    """Converts raw SoilGrids data into a DSSAT-formatted DataFrame."""
    logging.info("Processing raw data into DSSAT format...")
    dssat_df = pd.DataFrame(index=raw_df.index)
    for col in SOILGRIDS_PROPERTIES:
        if col not in raw_df or raw_df[col].isnull().all():
            logging.warning(f"Essential data column '{col}' is missing. DSSAT values will be NaN.")
            raw_df[col] = np.nan
    
    # SLB  - Layer Base Depth (cm)
    dssat_df['SLB'] = [int(label.split('-')[1].replace('cm', '')) for label in raw_df.index]
    # SBDM - Bulk Density, Moist (g/cm³)
    dssat_df['SBDM'] = raw_df['bdod'] / 100
    # SLCL - Clay Content (%)
    dssat_df['SLCL'] = raw_df['clay'] / 10
    # SLSI - Silt Content (%)
    dssat_df['SLSI'] = raw_df['silt'] / 10
    # SLCF - Coarse Fraction (>2mm) (%)
    dssat_df['SLCF'] = raw_df['cfvo'] / 10
    # SLLL - Lower Limit (Wilting Point) (cm³/cm³)
    dssat_df['SLLL'] = raw_df['wv1500'] / 1000
    # SDUL - Drained Upper Limit (Field Capacity) (cm³/cm³)
    dssat_df['SDUL'] = raw_df['wv0033'] / 1000
    # SSKS - Saturated Hydraulic Conductivity (cm/h)
    dssat_df['SSKS'] = np.nan
    # SSAT - Saturated Water Content (Porosity) (cm³/cm³)
    dssat_df['SSAT'] = 1 - (dssat_df['SBDM'] / PARTICLE_DENSITY_G_CM3)
    # SRGF - Root Growth Factor (0-1)
    rgf_map = {'0-5cm': 1.0, '5-15cm': 1.0, '15-30cm': 1.0, '30-60cm': 0.8, '60-100cm': 0.5, '100-200cm': 0.2}
    dssat_df['SRGF'] = dssat_df.index.map(rgf_map)
    # SLOC - Organic Carbon (%)
    dssat_df['SLOC'] = raw_df['soc'] / 100
    # SLHW - pH in Water
    dssat_df['SLHW'] = raw_df['phh2o'] / 10
    # SCEC - Cation Exchange Capacity (cmol/kg)
    dssat_df['SCEC'] = raw_df['cec'] / 10
    # SLNI - Total Nitrogen (%)
    dssat_df['SLNI'] = raw_df['nitrogen'] / 1000
    # SLPX - Phosphorus (not available)
    dssat_df['SLPX'] = np.nan
    # SLKE - Potassium (not available)
    dssat_df['SLKE'] = np.nan
    
    return dssat_df

def calculate_hydraulic_properties_with_rosetta(dssat_df: pd.DataFrame) -> pd.DataFrame:
    """Uses the Rosetta model to calculate SSKS."""
    logging.info("Calculating hydraulic properties with Rosetta model...")
    df = dssat_df.copy()
    df['SLSA'] = 100 - df['SLCL'] - df['SLSI']
    input_cols = ['SLSA', 'SLSI', 'SLCL', 'SBDM']
    if df[input_cols].isnull().values.any():
        logging.warning("Missing texture or bulk density. Cannot run Rosetta. SSKS will be NaN.")
        df['SSKS'] = np.nan
        return df.drop(columns=['SLSA'])
    try:
        soil_data = SoilData(sand=df['SLSA'].tolist(), silt=df['SLSI'].tolist(), clay=df['SLCL'].tolist(), bd=df['SBDM'].tolist())
        hydraulic_props = rosetta(3, soil_data)
        df['SSKS'] = [prop[4] / 24 for prop in hydraulic_props]
    except Exception as e:
        logging.error(f"Rosetta model failed: {e}. SSKS will be NaN.")
        df['SSKS'] = np.nan
    return df.drop(columns=['SLSA'])

def get_texture_class(sand: float, silt: float, clay: float) -> str:
    """Determines USDA soil texture class from sand, silt, clay percentages."""
    if (silt + 1.5 * clay) < 15: return 'S'
    if (silt + 1.5 * clay) >= 15 and (silt + 2 * clay) < 30: return 'LS'
    if (clay >= 7 and clay < 20) and (sand > 52) and ((silt + 2 * clay) >= 30): return 'SL'
    if (clay < 7) and (silt < 50) and ((silt + 2 * clay) >= 30): return 'SL'
    if (clay >= 7 and clay < 27) and (silt >= 28 and silt < 50) and (sand <= 52): return 'L'
    if (silt >= 50 and clay >= 12 and clay < 27): return 'SIL'
    if (silt >= 50 and clay < 12): return 'SI'
    if (silt >= 80 and clay < 12): return 'SI'
    if (clay >= 20 and clay < 35) and (silt < 28) and (sand > 45): return 'SCL'
    if (clay >= 27 and clay < 40) and (sand > 20 and sand <= 45): return 'CL'
    if (clay >= 27 and clay < 40) and (sand <= 20): return 'SICL'
    if (clay >= 35) and (sand > 45): return 'SC'
    if (clay >= 40) and (silt >= 40): return 'SIC'
    if (clay >= 40) and (sand <= 45) and (silt < 40): return 'C'
    return 'UNKNOWN'

def write_sol_file(df: pd.DataFrame, lat: float, lon: float, country: str, filename: str):
    """Writes the DataFrame to a DSSAT-compatible .SOL file."""
    logging.info(f"Writing DSSAT .SOL file to {filename}")
    top_layer = df.iloc[0]
    sand_pct = 100 - top_layer.get('SLCL', 0) - top_layer.get('SLSI', 0)
    texture = get_texture_class(sand_pct, top_layer.get('SLSI', 0), top_layer.get('SLCL', 0))
    soil_id = f"SG{str(int(abs(lat*100)))+str(int(abs(lon*100))):<10}"
    
    with open(filename, 'w') as f:
        f.write("*SOILS: DSSAT Soil Input File from SoilGrids Data\n!\n")
        f.write(f"! Auto-generated for location Lat={lat:.4f}, Lon={lon:.4f}\n!\n\n")
        f.write(f"*{soil_id} SOILGRIDS   {texture:<5s} {int(df['SLB'].max()):>5} SoilGrids Auto-Generated Profile\n")
        
        # --- CORRECTED @SITE LINE ---
        f.write("@SITE        COUNTRY          LAT     LONG SCS FAMILY\n")
        site_name = "SoilGrids"
        f.write(f" {site_name:<11s} {country:<15s} {lat:>8.2f} {lon:>8.2f} Generic\n")

        # --- DSSAT Master Parameter Glossary ---
        # SCOM - Soil Color (Not used)
        # SALB - Soil Albedo (0-1)
        # SLU1 - Stage 1 Evaporation Limit (mm)
        # SLDR - Soil Drainage Rate
        # SLRO - Runoff Curve Number
        # SLNF - Mineralization Factor (0-1)
        # SLPF - Photosynthesis Factor (0-1)
        # SMHB, SMPX, SMKE - Not used, placeholders for soil mineralogy
        f.write("@ SCOM  SALB  SLU1  SLDR  SLRO  SLNF  SLPF  SMHB  SMPX  SMKE\n")
        f.write("   -99  0.13   6.0   -99  75.0  1.00  1.00   -99   -99   -99\n")
        
        # --- DSSAT Soil Layer Parameter Glossary ---
        # SLB  - Layer Base Depth (cm)
        # SLMH - Master Horizon (-99 for not specified)
        # SLLL - Lower Limit (Wilting Point) (cm³/cm³)
        # SDUL - Drained Upper Limit (Field Capacity) (cm³/cm³)
        # SSAT - Saturated Water Content (Porosity) (cm³/cm³)
        # SRGF - Root Growth Factor (0-1)
        # SSKS - Saturated Hydraulic Conductivity (cm/h)
        # SBDM - Bulk Density, Moist (g/cm³)
        # SLOC - Organic Carbon (%)
        # SLCL - Clay Content (%)
        # SLSI - Silt Content (%)
        # SLCF - Coarse Fraction (>2mm) (%)
        # SLNI - Total Nitrogen (%)
        # SLHW - pH in Water
        # SLHB - Horizon Base (-99 for not specified)
        # SCEC - Cation Exchange Capacity (cmol/kg)
        # SADC - Anion Adsorption Coefficient (-99 for not specified)
        f.write("@  SLB  SLMH  SLLL  SDUL  SSAT  SRGF  SSKS  SBDM  SLOC  SLCL  SLSI  SLCF  SLNI  SLHW  SLHB  SCEC  SADC\n")
        
        for _, row in df.iterrows():
            row = row.fillna(-99)
            f.write(f"{row['SLB']:>6.0f} {-99:>5.0f} {row['SLLL']:>6.3f} {row['SDUL']:>6.3f} {row['SSAT']:>6.3f} {row['SRGF']:>6.3f} ")
            f.write(f"{row['SSKS']:>6.3f} {row['SBDM']:>6.3f} {row['SLOC']:>6.3f} {row['SLCL']:>6.1f} {row['SLSI']:>6.1f} ")
            f.write(f"{row['SLCF']:>6.1f} {row['SLNI']:>6.3f} {row['SLHW']:>6.1f} {-99:>5.0f} {row['SCEC']:>6.1f} {-99:>5.0f}\n")


# --- 3. MAIN EXECUTION LOGIC ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', stream=sys.stdout)

try:
    raw_soil_data = fetch_soilgrids_data_robust(LATITUDE, LONGITUDE, SOILGRIDS_PROPERTIES, SOILGRIDS_DEPTHS)
    dssat_profile = process_to_dssat_format(raw_soil_data)
    final_dssat_profile = calculate_hydraulic_properties_with_rosetta(dssat_profile)

    print("\n--- Final Processed DSSAT Data ---")
    print(final_dssat_profile.round(3).to_string())

    write_sol_file(final_dssat_profile, LATITUDE, LONGITUDE, COUNTRY, OUTPUT_FILENAME)
    print(f"\n✅ Successfully created soil file: {OUTPUT_FILENAME}")

except (requests.exceptions.RequestException, ValueError) as e:
    logging.error(f"Script failed: {e}")

In [ ]:
import logging
import sys
import time
from typing import List

import numpy as np
import pandas as pd
import requests
from rosetta import rosetta, SoilData

# --- 1. SET YOUR INPUTS HERE ---
#---------------------------------
LATITUDE = 12.3811
LONGITUDE = 78.9366
COUNTRY = "India"
OUTPUT_FILENAME = "MY_GENERATED_SOIL.SOL"
#---------------------------------


# --- 2. HELPER FUNCTIONS ---

# --- Configuration Constants ---
SOILGRIDS_DEPTHS = ["0-5cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm", "100-200cm"]
SOILGRIDS_PROPERTIES = [
    "sand", "silt", "clay", "bdod", "cfvo", "wv0033", "wv1500",
    "soc", "phh2o", "cec", "nitrogen"
]
PARTICLE_DENSITY_G_CM3 = 2.65

def fetch_soilgrids_data_robust(latitude: float, longitude: float, properties: List[str], depths: List[str]) -> pd.DataFrame:
    """Fetches soil property data robustly with a retry mechanism."""
    logging.info(f"Fetching SoilGrids data for lat={latitude}, lon={longitude}")
    base_url = "https://rest.isric.org/soilgrids/v2.0/properties/query"
    all_properties_data = []
    max_retries = 4
    initial_retry_delay_seconds = 5

    for prop in properties:
        logging.info(f"--> Fetching property: {prop}")
        params = {'lon': longitude, 'lat': latitude, 'property': prop, 'depth': depths, 'value': 'mean'}
        for attempt in range(max_retries):
            try:
                response = requests.get(base_url, params=params, timeout=45)
                if response.status_code == 429:
                    response.raise_for_status()
                response.raise_for_status()
                data = response.json()
                layers = data.get('properties', {}).get('layers', [])
                if not layers:
                    logging.warning(f"No layer data for {prop}. Skipping.")
                    break
                parsed_data = {
                    depth_data['label']: depth_data.get('values', {}).get('mean', np.nan)
                    for depth_data in layers[0]['depths']
                }
                prop_df = pd.DataFrame.from_dict(parsed_data, orient='index', columns=[prop])
                all_properties_data.append(prop_df)
                time.sleep(1)
                break
            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 429 and attempt < max_retries - 1:
                    wait_time = initial_retry_delay_seconds * (2 ** attempt)
                    logging.warning(f"Rate limit hit for '{prop}'. Retrying in {wait_time}s...")
                    time.sleep(wait_time)
                else:
                    logging.error(f"HTTP error for '{prop}': {e}. It will be missing.")
                    break
            except (requests.exceptions.RequestException, ValueError, KeyError) as e:
                logging.error(f"Failed on property '{prop}': {e}. Skipping.")
                break
    if not all_properties_data:
        raise ValueError("Failed to fetch any data from SoilGrids.")
    final_df = pd.concat(all_properties_data, axis=1).reindex(depths)
    for prop in properties:
        if prop not in final_df.columns:
            final_df[prop] = np.nan
    return final_df

def process_to_dssat_format(raw_df: pd.DataFrame) -> pd.DataFrame:
    """Converts raw SoilGrids data into a DSSAT-formatted DataFrame."""
    logging.info("Processing raw data into DSSAT format...")
    dssat_df = pd.DataFrame(index=raw_df.index)
    for col in SOILGRIDS_PROPERTIES:
        if col not in raw_df or raw_df[col].isnull().all():
            logging.warning(f"Essential data column '{col}' is missing. DSSAT values will be NaN.")
            raw_df[col] = np.nan
    
    # SLB  - Layer Base Depth (cm)
    dssat_df['SLB'] = [int(label.split('-')[1].replace('cm', '')) for label in raw_df.index]
    # SBDM - Bulk Density, Moist (g/cm³)
    dssat_df['SBDM'] = raw_df['bdod'] / 100
    # SLCL - Clay Content (%)
    dssat_df['SLCL'] = raw_df['clay'] / 10
    # SLSI - Silt Content (%)
    dssat_df['SLSI'] = raw_df['silt'] / 10
    # SLCF - Coarse Fraction (>2mm) (%)
    dssat_df['SLCF'] = raw_df['cfvo'] / 10
    # SLLL - Lower Limit (Wilting Point) (cm³/cm³)
    dssat_df['SLLL'] = raw_df['wv1500'] / 1000
    # SDUL - Drained Upper Limit (Field Capacity) (cm³/cm³)
    dssat_df['SDUL'] = raw_df['wv0033'] / 1000
    # SSKS - Saturated Hydraulic Conductivity (cm/h)
    dssat_df['SSKS'] = np.nan
    # SSAT - Saturated Water Content (Porosity) (cm³/cm³)
    dssat_df['SSAT'] = 1 - (dssat_df['SBDM'] / PARTICLE_DENSITY_G_CM3)
    # SRGF - Root Growth Factor (0-1)
    rgf_map = {'0-5cm': 1.0, '5-15cm': 1.0, '15-30cm': 1.0, '30-60cm': 0.8, '60-100cm': 0.5, '100-200cm': 0.2}
    dssat_df['SRGF'] = dssat_df.index.map(rgf_map)
    # SLOC - Organic Carbon (%)
    dssat_df['SLOC'] = raw_df['soc'] / 100
    # SLHW - pH in Water
    dssat_df['SLHW'] = raw_df['phh2o'] / 10
    # SCEC - Cation Exchange Capacity (cmol/kg)
    dssat_df['SCEC'] = raw_df['cec'] / 10
    # SLNI - Total Nitrogen (%)
    dssat_df['SLNI'] = raw_df['nitrogen'] / 1000
    # SLPX - Phosphorus (not available)
    dssat_df['SLPX'] = np.nan
    # SLKE - Potassium (not available)
    dssat_df['SLKE'] = np.nan
    
    return dssat_df

def calculate_hydraulic_properties_with_rosetta(dssat_df: pd.DataFrame) -> pd.DataFrame:
    """Uses the Rosetta model to calculate SSKS."""
    logging.info("Calculating hydraulic properties with Rosetta model...")
    df = dssat_df.copy()
    df['SLSA'] = 100 - df['SLCL'] - df['SLSI']
    input_cols = ['SLSA', 'SLSI', 'SLCL', 'SBDM']
    if df[input_cols].isnull().values.any():
        logging.warning("Missing texture or bulk density. Cannot run Rosetta. SSKS will be NaN.")
        df['SSKS'] = np.nan
        return df.drop(columns=['SLSA'])
    try:
        # MODIFIED: Prepare data as a list of lists first.
        rosetta_input_list = df[input_cols].values.tolist()

        # MODIFIED: Pass the list of lists to the SoilData constructor without keywords.
        soil_data = SoilData(rosetta_input_list)
        
        hydraulic_props = rosetta(3, soil_data)
        df['SSKS'] = [prop[4] / 24 for prop in hydraulic_props]
    except Exception as e:
        logging.error(f"Rosetta model failed: {e}. SSKS will be NaN.")
        df['SSKS'] = np.nan
    return df.drop(columns=['SLSA'])

def get_texture_class(sand: float, silt: float, clay: float) -> str:
    """Determines USDA soil texture class from sand, silt, clay percentages."""
    if (silt + 1.5 * clay) < 15: return 'S'
    if (silt + 1.5 * clay) >= 15 and (silt + 2 * clay) < 30: return 'LS'
    if (clay >= 7 and clay < 20) and (sand > 52) and ((silt + 2 * clay) >= 30): return 'SL'
    if (clay < 7) and (silt < 50) and ((silt + 2 * clay) >= 30): return 'SL'
    if (clay >= 7 and clay < 27) and (silt >= 28 and silt < 50) and (sand <= 52): return 'L'
    if (silt >= 50 and clay >= 12 and clay < 27): return 'SIL'
    if (silt >= 50 and clay < 12): return 'SI'
    if (silt >= 80 and clay < 12): return 'SI'
    if (clay >= 20 and clay < 35) and (silt < 28) and (sand > 45): return 'SCL'
    if (clay >= 27 and clay < 40) and (sand > 20 and sand <= 45): return 'CL'
    if (clay >= 27 and clay < 40) and (sand <= 20): return 'SICL'
    if (clay >= 35) and (sand > 45): return 'SC'
    if (clay >= 40) and (silt >= 40): return 'SIC'
    if (clay >= 40) and (sand <= 45) and (silt < 40): return 'C'
    return 'UNKNOWN'

def write_sol_file(df: pd.DataFrame, lat: float, lon: float, country: str, filename: str):
    """Writes the DataFrame to a DSSAT-compatible .SOL file."""
    logging.info(f"Writing DSSAT .SOL file to {filename}")
    top_layer = df.iloc[0]
    sand_pct = 100 - top_layer.get('SLCL', 0) - top_layer.get('SLSI', 0)
    texture = get_texture_class(sand_pct, top_layer.get('SLSI', 0), top_layer.get('SLCL', 0))
    soil_id = f"SG{str(int(abs(lat*100)))+str(int(abs(lon*100))):<10}"
    
    with open(filename, 'w') as f:
        f.write("*SOILS: DSSAT Soil Input File from SoilGrids Data\n!\n")
        f.write(f"! Auto-generated for location Lat={lat:.4f}, Lon={lon:.4f}\n!\n\n")
        f.write(f"*{soil_id} SOILGRIDS   {texture:<5s} {int(df['SLB'].max()):>5} SoilGrids Auto-Generated Profile\n")
        
        f.write("@SITE        COUNTRY          LAT     LONG SCS FAMILY\n")
        site_name = "SoilGrids"
        f.write(f" {site_name:<11s} {country:<15s} {lat:>8.2f} {lon:>8.2f} Generic\n")

        f.write("@ SCOM  SALB  SLU1  SLDR  SLRO  SLNF  SLPF  SMHB  SMPX  SMKE\n")
        f.write("   -99  0.13   6.0   -99  75.0  1.00  1.00   -99   -99   -99\n")
        
        f.write("@  SLB  SLMH  SLLL  SDUL  SSAT  SRGF  SSKS  SBDM  SLOC  SLCL  SLSI  SLCF  SLNI  SLHW  SLHB  SCEC  SADC\n")
        
        for _, row in df.iterrows():
            row = row.fillna(-99)
            f.write(f"{row['SLB']:>6.0f} {-99:>5.0f} {row['SLLL']:>6.3f} {row['SDUL']:>6.3f} {row['SSAT']:>6.3f} {row['SRGF']:>6.3f} ")
            f.write(f"{row['SSKS']:>6.3f} {row['SBDM']:>6.3f} {row['SLOC']:>6.3f} {row['SLCL']:>6.1f} {row['SLSI']:>6.1f} ")
            f.write(f"{row['SLCF']:>6.1f} {row['SLNI']:>6.3f} {row['SLHW']:>6.1f} {-99:>5.0f} {row['SCEC']:>6.1f} {-99:>5.0f}\n")


# --- 3. MAIN EXECUTION LOGIC ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', stream=sys.stdout)

try:
    raw_soil_data = fetch_soilgrids_data_robust(LATITUDE, LONGITUDE, SOILGRIDS_PROPERTIES, SOILGRIDS_DEPTHS)
    dssat_profile = process_to_dssat_format(raw_soil_data)
    final_dssat_profile = calculate_hydraulic_properties_with_rosetta(dssat_profile)

    print("\n--- Final Processed DSSAT Data ---")
    print(final_dssat_profile.round(3).to_string())

    write_sol_file(final_dssat_profile, LATITUDE, LONGITUDE, COUNTRY, OUTPUT_FILENAME)
    print(f"\n✅ Successfully created soil file: {OUTPUT_FILENAME}")

except (requests.exceptions.RequestException, ValueError) as e:
    logging.error(f"Script failed: {e}")

In [ ]:
import logging
import sys
import time
from typing import List

import numpy as np
import pandas as pd
import requests

!pip show dssattools

# Import the necessary classes AND the helper function from DSSATTools
from dssattools import SoilProfile, SoilLayer
from dssattools.soil import estimate_from_texture

# --- 1. SET YOUR INPUTS HERE ---
#---------------------------------
LATITUDE = 12.3811
LONGITUDE = 78.9366
COUNTRY = "India"
OUTPUT_FILENAME = "MY_GENERATED_SOIL.SOL"
#---------------------------------


# --- 2. HELPER FUNCTIONS ---

# --- Configuration Constants ---
SOILGRIDS_DEPTHS = ["0-5cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm", "100-200cm"]
SOILGRIDS_PROPERTIES = [
    "sand", "silt", "clay", "bdod", "cfvo", "wv0033", "wv1500",
    "soc", "phh2o", "cec", "nitrogen"
]

def fetch_soilgrids_data_robust(latitude: float, longitude: float, properties: List[str], depths: List[str]) -> pd.DataFrame:
    """Fetches soil property data robustly with a retry mechanism."""
    logging.info(f"Fetching SoilGrids data for lat={latitude}, lon={longitude}")
    base_url = "https://rest.isric.org/soilgrids/v2.0/properties/query"
    all_properties_data = []
    max_retries = 4
    initial_retry_delay_seconds = 5

    for prop in properties:
        logging.info(f"--> Fetching property: {prop}")
        params = {'lon': longitude, 'lat': latitude, 'property': prop, 'depth': depths, 'value': 'mean'}
        for attempt in range(max_retries):
            try:
                response = requests.get(base_url, params=params, timeout=45)
                response.raise_for_status()
                data = response.json()
                layers = data.get('properties', {}).get('layers', [])
                if not layers:
                    logging.warning(f"No layer data for {prop}. Skipping.")
                    break
                parsed_data = {
                    depth_data['label']: depth_data.get('values', {}).get('mean', np.nan)
                    for depth_data in layers[0]['depths']
                }
                prop_df = pd.DataFrame.from_dict(parsed_data, orient='index', columns=[prop])
                all_properties_data.append(prop_df)
                time.sleep(1)
                break
            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 429 and attempt < max_retries - 1:
                    wait_time = initial_retry_delay_seconds * (2 ** attempt)
                    logging.warning(f"Rate limit hit for '{prop}'. Retrying in {wait_time}s...")
                    time.sleep(wait_time)
                else:
                    logging.error(f"HTTP error for '{prop}': {e}. It will be missing.")
                    break
            except (requests.exceptions.RequestException, ValueError, KeyError) as e:
                logging.error(f"Failed on property '{prop}': {e}. Skipping.")
                break
    if not all_properties_data:
        raise ValueError("Failed to fetch any data from SoilGrids.")
    final_df = pd.concat(all_properties_data, axis=1).reindex(depths)
    for prop in properties:
        if prop not in final_df.columns:
            final_df[prop] = np.nan
    return final_df

# --- UPDATED processing function ---
def process_and_estimate_dssat_format(raw_df: pd.DataFrame) -> pd.DataFrame:
    """
    Converts raw SoilGrids data and uses the DSSATTools `estimate_from_texture` 
    function to calculate hydraulic properties for each layer.
    """
    logging.info("Processing raw data and estimating hydraulic properties with DSSATTools...")
    
    # Initialize DataFrame with basic info
    dssat_df = pd.DataFrame(index=raw_df.index)
    dssat_df['SLB'] = [int(label.split('-')[1].replace('cm', '')) for label in raw_df.index]
    dssat_df['SLOC'] = raw_df['soc'] / 100
    dssat_df['SLNI'] = raw_df['nitrogen'] / 1000
    dssat_df['SLHW'] = raw_df['phh2o'] / 10
    dssat_df['SCEC'] = raw_df['cec'] / 10
    dssat_df['SLCF'] = raw_df['cfvo'] / 10
    
    # Define root growth factor
    rgf_map = {'0-5cm': 1.0, '5-15cm': 1.0, '15-30cm': 1.0, '30-60cm': 0.8, '60-100cm': 0.5, '100-200cm': 0.2}
    dssat_df['SRGF'] = dssat_df.index.map(rgf_map)
    
    # Loop through each layer to estimate properties
    for index, row in raw_df.iterrows():
        slcl = row['clay'] / 10
        slsi = row['silt'] / 10
        sbdm = row['bdod'] / 100
        
        dssat_df.loc[index, 'SLCL'] = slcl
        dssat_df.loc[index, 'SLSI'] = slsi
        dssat_df.loc[index, 'SBDM'] = sbdm

        if pd.notna(slcl) and pd.notna(slsi) and pd.notna(sbdm):
            # Use the library's built-in function to get consistent properties
            estimated_props = estimate_from_texture(slcl=slcl, slsi=slsi, sbdm=sbdm)
            dssat_df.loc[index, 'SLLL'] = estimated_props['slll']
            dssat_df.loc[index, 'SDUL'] = estimated_props['sdul']
            dssat_df.loc[index, 'SSAT'] = estimated_props['ssat']
            dssat_df.loc[index, 'SSKS'] = estimated_props['ssks']
        else:
            logging.warning(f"Missing texture/bulk density for layer {index}. Cannot estimate properties.")

    return dssat_df

def get_texture_class(sand: float, silt: float, clay: float) -> str:
    """Determines USDA soil texture class from sand, silt, clay percentages."""
    if (silt + 1.5 * clay) < 15: return 'S'
    if (silt + 1.5 * clay) >= 15 and (silt + 2 * clay) < 30: return 'LS'
    if (clay >= 7 and clay < 20) and (sand > 52) and ((silt + 2 * clay) >= 30): return 'SL'
    if (clay < 7) and (silt < 50) and ((silt + 2 * clay) >= 30): return 'SL'
    if (clay >= 7 and clay < 27) and (silt >= 28 and silt < 50) and (sand <= 52): return 'L'
    if (silt >= 50 and clay >= 12 and clay < 27): return 'SIL'
    if (silt >= 50 and clay < 12): return 'SI'
    if (silt >= 80 and clay < 12): return 'SI'
    if (clay >= 20 and clay < 35) and (silt < 28) and (sand > 45): return 'SCL'
    if (clay >= 27 and clay < 40) and (sand > 20 and sand <= 45): return 'CL'
    if (clay >= 27 and clay < 40) and (sand <= 20): return 'SICL'
    if (clay >= 35) and (sand > 45): return 'SC'
    if (clay >= 40) and (silt >= 40): return 'SIC'
    if (clay >= 40) and (sand <= 45) and (silt < 40): return 'C'
    return 'UNKNOWN'

def write_sol_file_with_dssattools(df: pd.DataFrame, lat: float, lon: float, country: str, filename: str):
    """
    Builds a SoilProfile object from the DataFrame and writes it to a file
    using the DSSATTools library classes.
    """
    logging.info(f"Building SoilProfile object with DSSATTools...")
    
    soil_layers = []
    for _, row in df.iterrows():
        kwargs = {k.lower(): v for k, v in row.to_dict().items()}
        kwargs = {k: v for k, v in kwargs.items() if pd.notna(v)}
        soil_layers.append(SoilLayer(**kwargs))

    top_layer = df.iloc[0]
    sand_pct = 100 - top_layer.get('SLCL', 0) - top_layer.get('SLSI', 0)
    texture = get_texture_class(sand_pct, top_layer.get('SLSI', 0), top_layer.get('SLCL', 0))
    soil_id = f"SG{str(int(abs(lat*100))):0>4}{str(int(abs(lon*100))):0>4}"
    
    soil_profile = SoilProfile(
        name=soil_id,
        soil_series_name='SoilGrids Auto-Generated Profile',
        soil_clasification=texture,
        site='SoilGrids',
        country=country,
        lat=lat,
        long=lon,
        salb=0.13, slu1=6.0, slro=75.0, sldr=-99, slnf=1.0, slpf=1.0,
        table=soil_layers
    )
    
    logging.info(f"Writing DSSAT .SOL file to {filename}")
    with open(filename, 'w') as f:
        f.write(soil_profile._write_sol())


# --- 3. MAIN EXECUTION LOGIC ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', stream=sys.stdout)

try:
    raw_soil_data = fetch_soilgrids_data_robust(LATITUDE, LONGITUDE, SOILGRIDS_PROPERTIES, SOILGRIDS_DEPTHS)
    
    # Use the new, unified processing and estimation function
    final_dssat_profile = process_and_estimate_dssat_format(raw_soil_data)

    print("\n--- Final Processed DSSAT Data ---")
    print(final_dssat_profile.round(3).to_string())

    write_sol_file_with_dssattools(final_dssat_profile, LATITUDE, LONGITUDE, COUNTRY, OUTPUT_FILENAME)
    print(f"\n✅ Successfully created soil file using DSSATTools: {OUTPUT_FILENAME}")

    print(f"\nVerifying '{OUTPUT_FILENAME}'...")
    soil_id_to_check = f"SG{str(int(abs(LATITUDE*100))):0>4}{str(int(abs(LONGITUDE*100))):0>4}"
    verified_soil = SoilProfile.from_file(soil_id_to_check, OUTPUT_FILENAME)
    print("✅ Verification successful! The file can be read by DSSATTools.")

except Exception as e:
    logging.error(f"Script failed: {e}")